### Data collection for various metrics:

Data Collection for Safety and Stability Metrics (Ignore the efficiency metrics even though they are displayed). 

Collection performed in 3 stages

    A. Data collection for Safety and CAV (Controller Acceleration Variation) 
    B. Data collevtion for WAR (Wave Attenuation Ratio)
    C. Data collection for Stability Plot

### Notes:


- Desired velocity change for FS May need monitoring (May not be set for all possible lengths)
- PI can have failures, so have to turn ON render and manually check
- LACC needs a different shock time (for no shock change time at eval)
- Generated data will be saved in the folder `test_time_rollout`
- We apply shocks after the systems have been stabilized
- Default shock times are between 8000 and 11000 unless explicitly specified otherwise
- Difference: For stability plot, the length is 270 whereas for WAR, the length is 260
- For WAR num_rollours is set to 1

### Penetration rates
- num_controlled = 1 (5%), 4(20%), 9(40%), 13(16%) 

### Changes that need to be made to files: 

    1. 
    2. For Singleagent, Efficiency at test time, make the accel behavior zero from apply_rl_actions in density_aware_env (single agent version)
    3. For Multiagent, Efficiency at train time. Make sure the leader is correct in multiagent_ring. Make sure all test time markers are turned off. Make sure to have correct number of NUM_AUTOMATED at 2 places
    4. For Multiagent, Efficiency at test time, make sure the correct exp_configs are present in the same folder as test_rllib.py. in controllers for daware, set efficiency to True for TrainedAgent. 
    5. For Multiagent, Safety + Stability at test time, followers zero acceleration behavior is set from apply_rl_actions in density_aware_env (multiagent version)


### A and B together: (Data collection for Safety and CAV) + (Data collection for WAR)

In [1]:
# set parameters here
NUM_ROLLOUTS = 5
LENGTH = 260


### 1. (100%) IDM 

In [ ]:
!python classic.py --method idm --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render

In [7]:
!python eval_metrics.py --method idm --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/idm/idm_20240324-1117281711293448.7504716-0_emission.csv', './test_time_rollout/idm/idm_20240324-1119171711293557.020071-0_emission.csv', './test_time_rollout/idm/idm_20240324-1121061711293666.433222-0_emission.csv', './test_time_rollout/idm/idm_20240324-1122551711293775.7014508-0_emission.csv', './test_time_rollout/idm/idm_20240324-1124461711293886.0362644-0_emission.csv', './test_time_rollout/idm/idm_20240324-1126371711293997.819004-0_emission.csv', './test_time_rollout/idm/idm_20240324-1128321711294112.356284-0_emission.csv', './test_time_rollout/idm/idm_20240324-1130261711294226.9760046-0_emission.csv', './test_time_rollout/idm/idm_20240324-1132221711294342.5181315-0_emission.csv', './test_time_rollout/idm/idm_20240324-1134161711294456.9066727-0_emission.csv']

Vehicle ids: ['idm_0' 'idm_1' 'idm_2' 'idm_3' 'idm_4' 'idm_5' 'idm_6' 'idm_7' 'idm_8'
 'idm_9' 'idm_10' 'idm_11' 'idm_12' 'idm_13' 'idm_14' 'idm_15' 'idm_16

Total number of vehicles that crossed the zero point: 100

Flow (veh/hour): 1000.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[7.61310769 7.58158429 7.49656522 7.38640185 7.31079964 7.27332341
 7.2566489  7.29702392 7.36538895 7.4440374  7.5650437  7.65602597
 7.77354682 7.8355533  7.87158588 7.89263048 7.82465476 7.81075532
 7.79808861 7.78065104 7.72825982 7.67356933]

Miles per gallon (mpg): Average= 7.6, std = 0.21

####################
(22, 3600)

Average speed of each vehicle (m/s): 
[3.26759201 3.23311637 3.1981524  3.16691155 3.14865725 3.14855112
 3.16575929 3.19522979 3.23223543 3.27387938 3.31640312 3.35190453
 3.37393061 3.38401544 3.38671208 3.38629994 3.38354339 3.37683177
 3.36539765 3.34868485 3.32682795 3.29977513]

Standard deviation of speed of each vehicle (m/s): 
[3.60760411 3.58217717 3.56035932 3.53412247 3.51895085 3.51713156
 3.52705822 3.53260268 3.54187992 3.56840592 3.59269329 3.60604012
 3.59242295 3.58669394 3.581552   3.581

(22, 3600)

Average speed of each vehicle (m/s): 
[3.12099903 3.1303684  3.15132578 3.17947885 3.21312685 3.25200883
 3.29522967 3.3366361  3.3666951  3.38433335 3.39126408 3.38958095
 3.38280181 3.37112887 3.35430214 3.33079356 3.30309626 3.27011807
 3.23116964 3.18894949 3.15031299 3.12743482]

Standard deviation of speed of each vehicle (m/s): 
[3.50971555 3.51343368 3.51015964 3.51248708 3.50085387 3.50498374
 3.55154776 3.56060618 3.55383004 3.5408404  3.53186699 3.53655038
 3.59411863 3.6143855  3.61414513 3.61336623 3.61108419 3.60728779
 3.59225099 3.57625356 3.53786474 3.52308231]

Speed of all vehicles (m/s): Average= 3.27, std=3.56

####################
Total number of vehicles that crossed the zero point: 99

Flow (veh/hour): 990.0

MPG: [7.74353571 7.60076599 7.60160211 7.57808794 7.517961   7.71045695
 7.59408163 7.74668708 7.57813355 7.58147648], [0.21368735 0.2260338  0.21042853 0.23237051 0.23619797 0.23476693
 0.22280783 0.2700758  0.23737353 0.23088418]
#############

In [ ]:
# IDM does not need WAR, un-applicable

### 2. (5%) Single Vehicle Control

__FS__

In [ ]:
!python classic.py --method fs --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render --num_controlled 1

In [8]:
!python eval_metrics.py --method fs --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/fs/fs_20240324-1137581711294678.5075574-0_emission.csv', './test_time_rollout/fs/fs_20240324-1140241711294824.8041406-0_emission.csv', './test_time_rollout/fs/fs_20240324-1142441711294964.3615794-0_emission.csv', './test_time_rollout/fs/fs_20240324-1145021711295102.427448-0_emission.csv', './test_time_rollout/fs/fs_20240324-1147251711295245.7059307-0_emission.csv', './test_time_rollout/fs/fs_20240324-1149421711295382.5739076-0_emission.csv', './test_time_rollout/fs/fs_20240324-1151451711295505.8408105-0_emission.csv', './test_time_rollout/fs/fs_20240324-1153591711295639.2360945-0_emission.csv', './test_time_rollout/fs/fs_20240324-1156011711295761.5877073-0_emission.csv', './test_time_rollout/fs/fs_20240324-1158141711295894.7209227-0_emission.csv']

Vehicle ids: ['human_0' 'human_1' 'human_2' 'human_3' 'human_4' 'human_5' 'human_6'
 'human_7' 'human_8' 'human_9' 'human_10' 'human_11' 'human_12' 'human_13'
 'human_14' 'h

Total number of vehicles that crossed the zero point: 127

Flow (veh/hour): 1270.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[12.29289612 12.30838356 12.37196285 12.28128081 12.39819053 12.526906
 12.57341551 12.54142809 12.57405012 12.59661026 12.62627852 12.60185249
 12.65483834 12.46464353 12.32236863 11.99431948 11.77088498 11.70854041
 11.62651877 11.47478656 11.31003007 11.06707143]

Miles per gallon (mpg): Average= 12.19, std = 0.47

####################
(22, 3600)

Average speed of each vehicle (m/s): 
[4.31838306 4.3186718  4.31871599 4.31868084 4.3183648  4.31847557
 4.31876614 4.31887754 4.31894478 4.31884114 4.31902594 4.31882371
 4.31848104 4.31855168 4.31852685 4.31839776 4.31830839 4.31837754
 4.31842785 4.31861766 4.31835957 4.31860281]

Standard deviation of speed of each vehicle (m/s): 
[1.51917891 1.49139762 1.46275328 1.43417887 1.40244574 1.3679891
 1.33274768 1.29598885 1.25983625 1.21856229 1.17786306 1.13684767
 1.09427674 1.0527

Total number of vehicles that crossed the zero point: 127

Flow (veh/hour): 1270.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[12.92556912 12.86610606 12.91115473 13.01515332 12.86560236 12.86513064
 12.5738587  12.5853899  12.50248917 12.49061319 12.34632006 12.65388719
 12.588663   12.45366427 12.3140069  12.25527598 12.14233735 11.92266632
 11.83728554 11.58283957 11.44343552 10.97553958]

Miles per gallon (mpg): Average= 12.37, std = 0.52

####################
(22, 3600)

Average speed of each vehicle (m/s): 
[4.22117337 4.22067199 4.22040705 4.22036945 4.2207413  4.22101791
 4.22116741 4.22118348 4.22085797 4.22078145 4.22045684 4.2202228
 4.22028561 4.22046989 4.22067799 4.22079983 4.22084747 4.22071005
 4.22081505 4.22093037 4.22087097 4.22083952]

Standard deviation of speed of each vehicle (m/s): 
[1.80261715 1.76116629 1.71989812 1.68717959 1.64082875 1.59093912
 1.47302405 1.42929059 1.38383931 1.24043521 1.2008106  1.16832348
 1.12806204 1.09

In [ ]:
# Data for WAR
!python classic.py --method fs --stability --length $LENGTH --num_rollouts 1 --gen_emission --render --num_controlled 1

In [ ]:
# WAR
!python eval_plots.py --method fs 

__PIwS__
- This requires verifying if the particular run was `SUCCESS` or `FAIL`
- i.e., the controller may fail to stabilize the system

In [ ]:
# For CAV (first: get the data)
!python classic.py --method piws --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render --num_controlled 1

# Stable or not before shocks were applied
# 1. Stable 
# 2. Stable
# 3. Stable
# 4. Stable
# 5. Stable


In [11]:
# use the eval_metrics code
!python eval_metrics.py --method pi --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/pi/piws_20240324-1208141711296494.3419578-0_emission.csv', './test_time_rollout/pi/piws_20240324-1210261711296626.7265654-0_emission.csv', './test_time_rollout/pi/piws_20240324-1212381711296758.8740187-0_emission.csv', './test_time_rollout/pi/piws_20240324-1214491711296889.8857608-0_emission.csv', './test_time_rollout/pi/piws_20240324-1217081711297028.3604176-0_emission.csv', './test_time_rollout/pi/piws_20240324-1219281711297168.6246543-0_emission.csv', './test_time_rollout/pi/piws_20240324-1221471711297307.9739785-0_emission.csv', './test_time_rollout/pi/piws_20240324-1224181711297458.6881387-0_emission.csv', './test_time_rollout/pi/piws_20240324-1226391711297599.2852461-0_emission.csv', './test_time_rollout/pi/piws_20240324-1228581711297738.5415192-0_emission.csv']

Vehicle ids: ['human_0' 'human_1' 'human_2' 'human_3' 'human_4' 'human_5' 'human_6'
 'human_7' 'human_8' 'human_9' 'human_10' 'human_11' 'human_12' 'hum

Total number of vehicles that crossed the zero point: 124

Flow (veh/hour): 1240.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[13.98891566 14.02819684 14.13717897 14.22724296 14.14196861 14.15826776
 13.78215547 13.63661258 13.62524967 13.42684724 13.54969451 13.32394696
 13.24512416 13.01340671 12.85546323 12.75834161 12.6702777  12.50981421
 12.37845771 12.32809791 12.05736961 11.65191656]

Miles per gallon (mpg): Average= 13.25, std = 0.74

####################
(22, 3600)

Average speed of each vehicle (m/s): 
[4.48884123 4.48849632 4.48805684 4.48754835 4.48731956 4.48712203
 4.48710042 4.48684583 4.48662908 4.48696716 4.4870012  4.48719381
 4.48712293 4.48737975 4.48751321 4.48772684 4.48805926 4.48813593
 4.48808605 4.48796206 4.4877953  4.48778   ]

Standard deviation of speed of each vehicle (m/s): 
[1.52201311 1.48349648 1.44593352 1.4058192  1.36607898 1.31577157
 1.20831329 1.16716957 1.13258733 1.10567817 1.0392953  1.00337666
 0.96976365 0.9

Total number of vehicles that crossed the zero point: 131

Flow (veh/hour): 1310.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[13.64567963 13.55441637 13.60807921 13.6709552  13.43514786 13.44409574
 13.40725553 13.36708584 13.27585559 13.31799425 13.13231433 13.08401048
 12.96889132 13.01946883 13.03193411 12.86527964 12.57348956 12.51647056
 12.32163953 12.31787663 12.2789437  12.17720198]

Miles per gallon (mpg): Average= 13.05, std = 0.47

####################
(22, 3600)

Average speed of each vehicle (m/s): 
[4.71794148 4.71705397 4.71616745 4.71547003 4.71459288 4.71411289
 4.71366135 4.7135609  4.71324824 4.71311873 4.71321548 4.71350089
 4.71381036 4.71392284 4.71416469 4.71424168 4.71431857 4.71454622
 4.7148486  4.71499959 4.71498204 4.71512402]

Standard deviation of speed of each vehicle (m/s): 
[0.77491477 0.75939934 0.74020662 0.7186587  0.68784467 0.66876663
 0.63180553 0.60997967 0.59828478 0.58592526 0.57321492 0.55962613
 0.55079691 0.5

In [ ]:
# Data for WAR (Check if it stabilized or not)
!python classic.py --method piws --stability --length $LENGTH --num_rollouts 1 --gen_emission --render --num_controlled 1

# Stable or not before shocks were applied
# 1. 
# 2. 
# 3. 
# 4. 
# 5. 


In [ ]:
!python eval_plots.py --method pi 

#### BCM

In [ ]:
# For CAV (first get the data)
!python classic.py --method bcm --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 1

In [12]:
# use the eval_metrics code
!python eval_metrics.py --method bcm --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/bcm/bcm_20240324-1231201711297880.5168402-0_emission.csv', './test_time_rollout/bcm/bcm_20240324-1233341711298014.4494865-0_emission.csv', './test_time_rollout/bcm/bcm_20240324-1235491711298149.8013988-0_emission.csv', './test_time_rollout/bcm/bcm_20240324-1238051711298285.6715558-0_emission.csv', './test_time_rollout/bcm/bcm_20240324-1240161711298416.8434-0_emission.csv', './test_time_rollout/bcm/bcm_20240324-1242251711298545.0664306-0_emission.csv', './test_time_rollout/bcm/bcm_20240324-1244351711298675.1209254-0_emission.csv', './test_time_rollout/bcm/bcm_20240324-1246391711298799.9395163-0_emission.csv', './test_time_rollout/bcm/bcm_20240324-1248501711298930.4032292-0_emission.csv', './test_time_rollout/bcm/bcm_20240324-1250481711299048.8539073-0_emission.csv']

Vehicle ids: ['human_0' 'human_1' 'human_2' 'human_3' 'human_4' 'human_5' 'human_6'
 'human_7' 'human_8' 'human_9' 'human_10' 'human_11' 'human_12' 'human_

Total number of vehicles that crossed the zero point: 101

Flow (veh/hour): 1010.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[7.9329868  7.98707912 7.99147715 8.05205324 8.06490249 8.08085495
 8.10008087 8.1029566  8.100874   8.1329663  8.09531272 8.08776202
 8.10371099 8.13482762 8.1309541  8.1665826  8.23770738 8.26247562
 8.34316427 8.39905836 8.43940965 8.52567494]

Miles per gallon (mpg): Average= 8.16, std = 0.15

####################
(22, 3600)

Average speed of each vehicle (m/s): 
[3.41158885 3.42255527 3.43021842 3.43350264 3.43367384 3.43368164
 3.43369182 3.43371746 3.43369937 3.43372168 3.43272464 3.43016392
 3.42680387 3.42066226 3.41600649 3.41116457 3.4056518  3.40088219
 3.39743626 3.39449878 3.39554782 3.40117209]

Standard deviation of speed of each vehicle (m/s): 
[3.59497851 3.58948371 3.5812619  3.56405875 3.54970782 3.53524415
 3.51784293 3.49725962 3.48569825 3.47592735 3.4711737  3.50523931
 3.48853377 3.47623419 3.46018144 3.43

(22, 3600)

Average speed of each vehicle (m/s): 
[3.4356045  3.43599466 3.43580253 3.43653629 3.43653344 3.43840682
 3.43817655 3.43931183 3.43869758 3.43725031 3.43539741 3.43422948
 3.43391188 3.43392225 3.43390861 3.43386339 3.4338836  3.43390942
 3.43389745 3.43414247 3.43469942 3.43517002]

Standard deviation of speed of each vehicle (m/s): 
[3.61091605 3.61364073 3.61425008 3.60804768 3.60206008 3.59821777
 3.59075515 3.58897122 3.58501394 3.5871422  3.57486161 3.56392414
 3.54762561 3.51796207 3.49237521 3.46011803 3.42679289 3.38907439
 3.3530229  3.31753003 3.28269682 3.24910037]

Speed of all vehicles (m/s): Average= 3.44, std=3.51

####################
Total number of vehicles that crossed the zero point: 105

Flow (veh/hour): 1050.0

MPG: [8.15299483 8.24992288 8.15785781 8.0455541  8.26573915 8.03956171
 8.09282738 8.43676723 8.13619871 8.14307114], [0.16881265 0.0860522  0.14784575 0.2294608  0.23572408 0.25284217
 0.16136134 0.23836787 0.16871015 0.19550121]
###########

In [ ]:
# No WAR. BCM is Unstable. But useful for later, to estimate in bottleneck.
!python classic.py --method bcm --stability --length $LENGTH --gen_emission\
--num_rollouts 5 --render --num_controlled 1

In [ ]:
!python eval_plots.py --method bcm 

#### LACC

In [ ]:
# For CAV (first get the data)
!python classic.py --method lacc --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --shock_start_time 11400 --shock_end_time 15000 --num_controlled 1


In [13]:
# use the eval_metrics code
!python eval_metrics.py --method lacc --start_time 11400 --end_time 15000 --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/lacc/lacc_20240324-1216071711296967.0201242-0_emission.csv', './test_time_rollout/lacc/lacc_20240324-1218201711297100.2132988-0_emission.csv', './test_time_rollout/lacc/lacc_20240324-1220321711297232.5002747-0_emission.csv', './test_time_rollout/lacc/lacc_20240324-1222501711297370.1241796-0_emission.csv', './test_time_rollout/lacc/lacc_20240324-1225131711297513.0213444-0_emission.csv', './test_time_rollout/lacc/lacc_20240324-1227261711297646.0951104-0_emission.csv', './test_time_rollout/lacc/lacc_20240324-1229401711297780.4153874-0_emission.csv', './test_time_rollout/lacc/lacc_20240324-1231541711297914.447354-0_emission.csv', './test_time_rollout/lacc/lacc_20240324-1234091711298049.3870497-0_emission.csv', './test_time_rollout/lacc/lacc_20240324-1236261711298186.199213-0_emission.csv']

Vehicle ids: ['human_0' 'human_1' 'human_2' 'human_3' 'human_4' 'human_5' 'human_6'
 'human_7' 'human_8' 'human_9' 'human_10' 'human_1

Total number of vehicles that crossed the zero point: 104

Flow (veh/hour): 1040.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[8.01658811 8.0263677  8.03650132 8.07334241 8.07023403 8.01337984
 8.00099074 8.09043396 8.11379932 8.15926582 8.20887339 8.23443492
 8.25311651 8.26412864 8.27887861 8.23053957 8.23949868 8.2554863
 8.17293408 8.14445834 8.20696764 8.23897056]

Miles per gallon (mpg): Average= 8.15, std = 0.09

####################
(22, 3600)

Average speed of each vehicle (m/s): 
[3.40093052 3.39212168 3.38271294 3.37439818 3.36571856 3.36535214
 3.37435696 3.3910363  3.409103   3.4255791  3.43353388 3.43566853
 3.4356748  3.43571522 3.43564935 3.4356444  3.43560869 3.4342615
 3.42998714 3.42386959 3.4169471  3.40898162]

Standard deviation of speed of each vehicle (m/s): 
[3.60087089 3.57114481 3.53440614 3.49990729 3.46243613 3.43528901
 3.46097386 3.45414165 3.44688087 3.4245135  3.40241348 3.40439489
 3.38714188 3.36711204 3.35039796 3.3328

(22, 3600)

Average speed of each vehicle (m/s): 
[3.43448927 3.43344907 3.42930852 3.42144897 3.41226841 3.40211431
 3.39176829 3.38097648 3.37003688 3.36142451 3.3529194  3.35329793
 3.36594583 3.38673802 3.40742412 3.42396325 3.43353446 3.43728768
 3.43442527 3.43445936 3.43448612 3.43449566]

Standard deviation of speed of each vehicle (m/s): 
[3.59265783 3.58455403 3.57738555 3.59364623 3.58459915 3.57560778
 3.56446277 3.54984417 3.52630608 3.4956186  3.46694829 3.45801182
 3.48321251 3.49270831 3.4859996  3.4693859  3.49540381 3.52183222
 3.52554276 3.51793396 3.50411731 3.49558153]

Speed of all vehicles (m/s): Average= 3.41, std=3.53

####################
Total number of vehicles that crossed the zero point: 100

Flow (veh/hour): 1000.0

MPG: [7.98804291 7.90573491 8.15132684 8.01644865 7.91176199 8.08557709
 8.01391206 7.97592591 8.0059669  8.0054788 ], [0.13363497 0.14332335 0.0934647  0.04644849 0.13134789 0.13466363
 0.1435994  0.16155972 0.14831291 0.08840233]
###########

In [ ]:
# No WAR. LACC is Unstable. But useful for later, to estimate in bottleneck.
# Data for WAR
!python classic.py --method lacc --stability --length $LENGTH --num_rollouts 5 --gen_emission\
--render --num_controlled 1 --shock_start_time 11400 --shock_end_time 15000

In [ ]:
# WAR
!python eval_plots.py --method lacc --start_time 11400 --end_time 15000 --propogate_time 200

__Wu et. al.__

In [ ]:
# For CAV (first get the data)
!python test_rllib.py ./Wu_et_al/Trained_policies/5_percent/PPO_WaveAttenuationPOEnv-v0_25b5cb6e_2022-01-26_10-58-12e9f4i3ao 50 \
--method wu --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 1


In [14]:
# For CAV
!python eval_metrics.py --method wu --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/wu/stabilizing_the_ring_20240324-1357341711303054.5021448-0_emission.csv', './test_time_rollout/wu/stabilizing_the_ring_20240324-1400091711303209.781603-0_emission.csv', './test_time_rollout/wu/stabilizing_the_ring_20240324-1402471711303367.0658877-0_emission.csv', './test_time_rollout/wu/stabilizing_the_ring_20240324-1405351711303535.6004627-0_emission.csv', './test_time_rollout/wu/stabilizing_the_ring_20240324-1407571711303677.0139954-0_emission.csv', './test_time_rollout/wu/stabilizing_the_ring_20240324-1410161711303816.3475657-0_emission.csv', './test_time_rollout/wu/stabilizing_the_ring_20240324-1412591711303979.9636114-0_emission.csv', './test_time_rollout/wu/stabilizing_the_ring_20240324-1415191711304119.7829416-0_emission.csv', './test_time_rollout/wu/stabilizing_the_ring_20240324-1418011711304281.2087781-0_emission.csv', './test_time_rollout/wu/stabilizing_the_ring_20240324-1420211711304421.788183-0_emission.c

Total number of vehicles that crossed the zero point: 103

Flow (veh/hour): 1030.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[11.07915396 11.26273224 11.360167   11.35979421 11.43489491 11.46070964
 11.38493396 11.23657966 11.14342687 10.88188679 10.5520814  10.31395842
  9.85535178  9.37272786  9.19437344  9.11929778  8.87167112  8.59712072
  8.39893068  8.31531623  8.2743801   8.8251062 ]

Miles per gallon (mpg): Average= 10.1, std = 1.19

####################
(22, 3600)

Average speed of each vehicle (m/s): 
[3.39167993 3.38030436 3.36944009 3.35902794 3.34943515 3.34049957
 3.33226723 3.32386428 3.3143831  3.30363619 3.29227669 3.28129768
 3.27089814 3.26147636 3.25267335 3.24466395 3.23742971 3.23071686
 3.2240406  3.21683143 3.20911289 3.20181038]

Standard deviation of speed of each vehicle (m/s): 
[1.92941548 1.90516259 1.88288297 1.8576841  1.83149651 1.80301702
 1.77025851 1.73531983 1.70146215 1.66274032 1.633235   1.58223197
 1.45127275 1.38

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[11.66927001 11.64362035 11.67687021 11.67463818 11.63831386 11.42095293
 11.01018774 10.76227951 10.46431295 10.16945242 10.06380198  9.88352389
  9.81153717  9.71932064  9.62415168  9.55805426  9.38723518  9.26909238
  9.23680242  9.09114109  8.94344495  9.15891901]

Miles per gallon (mpg): Average= 10.27, std = 0.97

####################
(22, 3600)

Average speed of each vehicle (m/s): 
[3.5593045  3.54957958 3.53929671 3.52955069 3.5201614  3.51117231
 3.5028909  3.49456759 3.4857723  3.47670763 3.46749405 3.45839493
 3.44973414 3.44195377 3.4348726  3.42832003 3.42201142 3.41585527
 3.40938729 3.40400649 3.39890647 3.3936304 ]

Standard deviation of speed of each vehicle (m/s): 
[1.72056045 1.6844477  1.64590744 1.60278721 1.55865437 1.50767919
 1.32440204 1.29210805 1.24805259 1.19295616 1.16393111 1.15313467
 1.14179317 1.13009895 1.11891465 1.1071776  1.07991346 1.06773781
 1.05719412 1.03335987 1.0255273

In [ ]:
# Data for WAR
!python test_rllib.py ./Wu_et_al/Trained_policies/5_percent/PPO_WaveAttenuationPOEnv-v0_25b5cb6e_2022-01-26_10-58-12e9f4i3ao 50\
--method wu --shock --stability --length $LENGTH --gen_emission --num_rollouts 1 --render --num_controlled 1


In [ ]:
!python eval_plots.py --method wu 

__Ours (1x)__
- The local zone is set to 50m at training. And it is set to 50m here as well.

In [ ]:
# For CAV (Safety + Stability)
!python test_rllib.py ./Ours/Trained_policies/5_percent/PPO_DensityAwareRLEnv-v0_5dfded14_2024-02-07_09-58-351869u4p3 168\
--method ours --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 1


In [17]:
# Only look at safety and stability
!python eval_metrics.py --method ours --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/ours/density_aware_rl_20240326-1233321711470812.9831192-0_emission.csv', './test_time_rollout/ours/density_aware_rl_20240326-1233471711470827.6947434-0_emission.csv', './test_time_rollout/ours/density_aware_rl_20240326-1236131711470973.1215453-0_emission.csv', './test_time_rollout/ours/density_aware_rl_20240326-1236301711470990.0804913-0_emission.csv', './test_time_rollout/ours/density_aware_rl_20240326-1239041711471144.5393755-0_emission.csv', './test_time_rollout/ours/density_aware_rl_20240326-1239231711471163.415618-0_emission.csv', './test_time_rollout/ours/density_aware_rl_20240326-1241521711471312.600581-0_emission.csv', './test_time_rollout/ours/density_aware_rl_20240326-1242141711471334.322734-0_emission.csv', './test_time_rollout/ours/density_aware_rl_20240326-1244351711471475.5918193-0_emission.csv', './test_time_rollout/ours/density_aware_rl_20240326-1245001711471500.8275692-0_emission.csv']

Vehicle ids: ['

Total number of vehicles that crossed the zero point: 130

Flow (veh/hour): 1300.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[13.78477357 13.75759341 13.78755304 13.76087774 13.85371957 13.94579204
 14.09022693 13.98469017 13.6346031  13.5917773  13.48496168 13.00179837
 12.83797233 12.68184753 12.39545963 12.25043495 12.07343037 11.88675757
 11.65648992 11.44319014 11.28013367 12.44871237]

Miles per gallon (mpg): Average= 12.98, std = 0.9

####################
(22, 3600)

Average speed of each vehicle (m/s): 
[4.12379389 4.13427937 4.14183664 4.14600548 4.14659178 4.14362021
 4.13795311 4.12913687 4.11808132 4.10517033 4.09222253 4.08038412
 4.06960043 4.06137715 4.05607047 4.05263251 4.05142956 4.0518475
 4.05324511 4.05557934 4.05947897 4.06440142]

Standard deviation of speed of each vehicle (m/s): 
[1.92223726 1.87494495 1.84756569 1.78656741 1.75378651 1.64759474
 1.49775019 1.36138066 1.33527741 1.29313918 1.26777573 1.24274133
 1.19922163 1.175

Total number of vehicles that crossed the zero point: 126

Flow (veh/hour): 1260.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[13.39536476 13.61257465 13.84266762 13.89119339 13.88077582 13.59865174
 13.35468352 13.29809331 13.2666653  13.1525616  13.25854523 13.247963
 13.03471393 12.86792676 12.5552134  12.52744778 12.30359207 12.15878157
 12.1148923  11.85391621 11.65261202 12.59093184]

Miles per gallon (mpg): Average= 12.98, std = 0.65

####################
(22, 3600)

Average speed of each vehicle (m/s): 
[4.11546586 4.11902073 4.12452982 4.13131228 4.13914364 4.14732977
 4.15502517 4.16131822 4.16551675 4.16779438 4.16805938 4.16710953
 4.16472972 4.16193507 4.15820138 4.15311436 4.1463658  4.13805993
 4.12926712 4.12182802 4.11689632 4.11512648]

Standard deviation of speed of each vehicle (m/s): 
[1.80993839 1.78932851 1.76804558 1.74591899 1.73089734 1.70663352
 1.69678511 1.68884522 1.7098024  1.67481813 1.62394986 1.57367876
 1.5189588  1.392

In [ ]:
# For CAV (Efficiency)

!python test_rllib.py ./Ours/Trained_policies/5_percent/PPO_DensityAwareRLEnv-v0_6106dcf6_2024-02-08_12-42-07_7vnf1n4 200\
--method ours --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 1

In [ ]:
# Only look at Efficiency (Wipe out the previous data)
!python eval_metrics.py --method ours --save_plots

In [ ]:
# Data for WAR (Take the Safety + Stability Policy)
!python test_rllib.py ./Ours/Trained_policies/5_percent/PPO_DensityAwareRLEnv-v0_5dfded14_2024-02-07_09-58-351869u4p3 168\
--method ours --shock --stability --length $LENGTH --gen_emission --num_rollouts 1 --render --num_controlled 1


In [ ]:
!python eval_plots.py --method ours

### 3. (20%) Multiple Vehicle Control

### FS

In [ ]:
!python classic.py --method fs --length $LENGTH --gen_emission --num_rollouts\
$NUM_ROLLOUTS --shock --render --num_controlled 4


In [18]:
# use the eval_metrics code
!python eval_metrics.py --method fs --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/fs/fs_20240326-1107501711465670.4257193-0_emission.csv', './test_time_rollout/fs/fs_20240326-1110131711465813.234852-0_emission.csv', './test_time_rollout/fs/fs_20240326-1112441711465964.4365945-0_emission.csv', './test_time_rollout/fs/fs_20240326-1115091711466109.1223695-0_emission.csv', './test_time_rollout/fs/fs_20240326-1117151711466235.7948072-0_emission.csv', './test_time_rollout/fs/fs_20240326-1119101711466350.7786646-0_emission.csv', './test_time_rollout/fs/fs_20240326-1121161711466476.0246472-0_emission.csv', './test_time_rollout/fs/fs_20240326-1123211711466601.6710212-0_emission.csv', './test_time_rollout/fs/fs_20240326-1125341711466734.5062046-0_emission.csv', './test_time_rollout/fs/fs_20240326-1127291711466849.7427928-0_emission.csv']

Vehicle ids: ['human_0' 'human_1' 'human_2' 'human_3' 'human_4' 'human_5' 'human_6'
 'human_7' 'human_8' 'human_9' 'human_10' 'human_11' 'human_12' 'human_13'
 'human_14' 'h

(4,)
Worst case drac for each vehicle (m/s^2): 
[0.008681   0.00334566 0.00521501 0.78123525]

DRAC (m/s^2):
	Worst= 0.78

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[13.1872252  13.27201972 13.27026676 13.31612295 13.18250396 12.97251664
 12.74985135 12.33435262 12.1642085  11.9772956  11.83445769 11.80914369
 11.79052252 11.71705349 11.64416275 11.62081996 11.49460543 11.38227875
 11.23053875 11.23027523 11.22496341 11.21478692]

Miles per gallon (mpg): Average= 12.12, std = 0.76

####################
(22, 3600)

Average speed of each vehicle (m/s): 
[4.45070957 4.45072007 4.4508028  4.45091008 4.4508555  4.45098512
 4.45120754 4.45123793 4.45132451 4.4511416  4.45101117 4.45072412
 4.45070631 4.451256   4.45139562 4.45154212 4.45153042 4.45164792
 4.45150756 4.45151657 4.45152379 4.45152975]

Standard deviation of speed of each vehicle (m/s): 
[1.1962533  1.16249905 1.13052851 1.09253054 1.05573602 1.00500421
 0.95034245 0.67366162 0.65559451 0.6098046

Total number of vehicles that crossed the zero point: 135

Flow (veh/hour): 1350.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[12.83967765 12.76651108 12.68573778 12.66367529 12.57632156 12.57415617
 12.38113779 12.32978282 12.25794339 12.18527252 12.15239409 12.02799132
 11.79262631 11.76497986 11.70515489 11.58000555 11.44309799 11.40567417
 11.2699881  11.27613941 11.27486536 11.26750027]

Miles per gallon (mpg): Average= 12.01, std = 0.54

####################
(22, 3600)

Average speed of each vehicle (m/s): 
[4.49390905 4.49351947 4.4933325  4.49355714 4.49395109 4.49420217
 4.49457937 4.49461517 4.49461046 4.49439947 4.49443417 4.49440563
 4.49443234 4.49426153 4.49417694 4.49414623 4.49440452 4.49433266
 4.49423098 4.49426853 4.494307   4.49434597]

Standard deviation of speed of each vehicle (m/s): 
[0.95674789 0.78460777 0.77113952 0.7497031  0.72833963 0.70584838
 0.64916209 0.62648619 0.60495711 0.58498333 0.56010133 0.53958568
 0.41075739 0.4

####################
Acceleration variation (m/s^2): [0.22543593147761126, 0.36912642266685847, 0.25401463620212766, 0.10244401172449576, 0.2157830054967925, 0.2274650944734257, 0.2776572315822347]

File: ./test_time_rollout/fs/fs_20240326-1125341711466734.5062046-0_emission.csv
(4, 3600)
Worst case time headway of controller(s): 
[1.16128293 1.16139361 1.16182556 1.17431779]


Average time headway of controller(s): 
[1.21634657 1.21529201 1.21497224 3.30692227]

Standard deviation of time headway of controller(s): 
[0.27540355 0.26353675 0.25726132 0.81540694]

Time headway (s): Worst = 1.16, Avg= 1.74, std= 0.4

####################
####################
Acceleration variation (m/s^2): [0.22543593147761126, 0.36912642266685847, 0.25401463620212766, 0.10244401172449576, 0.2157830054967925, 0.2274650944734257, 0.2776572315822347, 0.197032707351915]

File: ./test_time_rollout/fs/fs_20240326-1127291711466849.7427928-0_emission.csv
(4, 3600)
Worst case time headway of controller(s): 
[1.16

In [ ]:
# Data for WAR. Desired velocity should be set to 4.82 here
!python classic.py --method fs --stability --length $LENGTH --num_rollouts 5 --gen_emission\
--render --num_controlled 4


In [ ]:
# WAR
!python eval_plots.py --method fs 

__PIwS__

In [ ]:
!python classic.py --method piws --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS\
--shock --render --num_controlled 4


In [19]:
!python eval_metrics.py --method pi --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/pi/piws_20240326-1130561711467056.2862604-0_emission.csv', './test_time_rollout/pi/piws_20240326-1133191711467199.129448-0_emission.csv', './test_time_rollout/pi/piws_20240326-1135411711467341.6869388-0_emission.csv', './test_time_rollout/pi/piws_20240326-1138021711467482.728351-0_emission.csv', './test_time_rollout/pi/piws_20240326-1140221711467622.5334115-0_emission.csv', './test_time_rollout/pi/piws_20240326-1142421711467762.5642595-0_emission.csv', './test_time_rollout/pi/piws_20240326-1145011711467901.0223098-0_emission.csv', './test_time_rollout/pi/piws_20240326-1147041711468024.4272783-0_emission.csv', './test_time_rollout/pi/piws_20240326-1149151711468155.6095147-0_emission.csv', './test_time_rollout/pi/piws_20240326-1151281711468288.6914616-0_emission.csv']

Vehicle ids: ['human_0' 'human_1' 'human_2' 'human_3' 'human_4' 'human_5' 'human_6'
 'human_7' 'human_8' 'human_9' 'human_10' 'human_11' 'human_12' 'human

(22, 3600)

Average speed of each vehicle (m/s): 
[4.38642208 4.38575431 4.38483638 4.38367819 4.38273507 4.3816151
 4.38029748 4.37885252 4.37764748 4.37608334 4.37478171 4.37357706
 4.37268872 4.37172117 4.37033941 4.36894009 4.36753599 4.36629343
 4.36494197 4.36873286 4.37439444 4.38375062]

Standard deviation of speed of each vehicle (m/s): 
[1.23461878 1.18784767 1.13976914 1.03258878 1.00028708 0.88694291
 0.70845719 0.68741473 0.66850802 0.66257578 0.64966521 0.5785664
 0.56310262 0.55168196 0.53634903 0.52275347 0.51426262 0.41208292
 0.4109966  0.4437648  0.44824625 0.59604227]

Speed of all vehicles (m/s): Average= 4.38, std=0.7

####################
Total number of vehicles that crossed the zero point: 133

Flow (veh/hour): 1330.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[12.02305537 11.90491765 11.59974977 11.55687886 11.48234673 11.23293814
 11.2087935  11.12234715 11.10891082 11.02260917 11.0015538  10.84111707
 10.66188133 10.62107776 1

(22, 3600)

Average speed of each vehicle (m/s): 
[4.232427   4.23013584 4.22797521 4.22606354 4.22441844 4.22248528
 4.2208808  4.21962924 4.21852386 4.21725343 4.21591081 4.21489919
 4.21358429 4.21271588 4.21184548 4.21120923 4.21026689 4.20945212
 4.20855648 4.23490776 4.24163304 4.23416311]

Standard deviation of speed of each vehicle (m/s): 
[1.22718537 1.19660289 1.08846208 1.06328337 1.03431701 0.99960426
 0.9680946  0.93799851 0.90713957 0.87375076 0.73335767 0.66851444
 0.65473857 0.59811477 0.58921869 0.56867224 0.56150257 0.55614958
 0.47991589 0.62128441 0.65349122 0.68311066]

Speed of all vehicles (m/s): Average= 4.22, std=0.8

####################
Total number of vehicles that crossed the zero point: 129

Flow (veh/hour): 1290.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[12.88155311 12.84765775 12.63450106 12.51903432 12.30104082 12.53961451
 12.44281537 12.43045976 12.28825048 12.06513562 11.88039609 11.68915364
 11.63565173 11.64517856

####################
Acceleration variation (m/s^2): [0.2526381039414621, 0.25829492867097664, 0.21290738748346638, 0.23085694796752776, 0.18017748686663412, 0.2031569273150201, 0.12512518362322836, 0.2018043827808282]

File: ./test_time_rollout/pi/piws_20240326-1151281711468288.6914616-0_emission.csv
(4, 3600)
Worst case time headway of controller(s): 
[1.02100529 1.00623472 1.01616912 1.02186365]


Average time headway of controller(s): 
[1.69192896 1.79914702 1.99563314 2.17089341]

Standard deviation of time headway of controller(s): 
[0.72699392 0.85305798 0.96782665 1.2355528 ]

Time headway (s): Worst = 1.01, Avg= 1.91, std= 0.95

####################
####################
Acceleration variation (m/s^2): [0.2526381039414621, 0.25829492867097664, 0.21290738748346638, 0.23085694796752776, 0.18017748686663412, 0.2031569273150201, 0.12512518362322836, 0.2018043827808282, 0.21775735404729668]

####################
####################

Final Aggregated Safety Metrics (across all files

In [ ]:
# Data for WAR (Check if it stabilized or not)
!python classic.py --method piws --stability --length $LENGTH --num_rollouts 5 --gen_emission\
--render --num_controlled 4


In [ ]:
!python eval_plots.py --method pi 

__BCM__

In [ ]:
# For CAV (first get the data)
!python classic.py --method bcm --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 4

In [20]:
# use the eval_metrics code
!python eval_metrics.py --method bcm --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/bcm/bcm_20240326-1154461711468486.6950676-0_emission.csv', './test_time_rollout/bcm/bcm_20240326-1157111711468631.96163-0_emission.csv', './test_time_rollout/bcm/bcm_20240326-1159351711468775.8905528-0_emission.csv', './test_time_rollout/bcm/bcm_20240326-1202021711468922.6832883-0_emission.csv', './test_time_rollout/bcm/bcm_20240326-1204321711469072.4478478-0_emission.csv', './test_time_rollout/bcm/bcm_20240326-1206471711469207.9703362-0_emission.csv', './test_time_rollout/bcm/bcm_20240326-1208531711469333.859267-0_emission.csv', './test_time_rollout/bcm/bcm_20240326-1211011711469461.4063885-0_emission.csv', './test_time_rollout/bcm/bcm_20240326-1213131711469593.8931115-0_emission.csv', './test_time_rollout/bcm/bcm_20240326-1215321711469732.3578181-0_emission.csv']

Vehicle ids: ['human_0' 'human_1' 'human_2' 'human_3' 'human_4' 'human_5' 'human_6'
 'human_7' 'human_8' 'human_9' 'human_10' 'human_11' 'human_12' 'human_

(22, 3600)

Average speed of each vehicle (m/s): 
[4.62009282 4.62078054 4.62134237 4.62149115 4.62115666 4.6211134
 4.62096945 4.62078097 4.62044733 4.61982705 4.61922679 4.61884618
 4.61838665 4.618126   4.61799627 4.6178921  4.61776021 4.61757468
 4.61763794 4.61775378 4.61806145 4.61876421]

Standard deviation of speed of each vehicle (m/s): 
[1.34497557 1.3199047  1.29379023 1.25330993 1.21839248 1.18732248
 1.12659931 1.07978034 1.04643232 0.9935665  0.9562792  0.92512383
 0.88898799 0.84699737 0.68137422 0.6611684  0.64373218 0.62915924
 0.60474782 0.65175901 0.78421052 1.01135415]

Speed of all vehicles (m/s): Average= 4.62, std=0.96

####################
Total number of vehicles that crossed the zero point: 141

Flow (veh/hour): 1410.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[14.08346848 13.95087585 13.78902222 13.76775195 13.64510823 13.63195334
 13.63320529 13.57070078 13.5894058  13.53505287 13.60674719 13.54212048
 13.22415374 13.10497522

(22, 3600)

Average speed of each vehicle (m/s): 
[4.50402547 4.50335977 4.50245504 4.50203368 4.50150071 4.5011844
 4.50045016 4.50014757 4.49971984 4.49951556 4.49942024 4.49955662
 4.49958777 4.4996724  4.50005105 4.50044428 4.50086687 4.50132489
 4.50156076 4.50203433 4.5026923  4.50342248]

Standard deviation of speed of each vehicle (m/s): 
[1.67944558 1.63663165 1.59928102 1.58560914 1.54369529 1.50106677
 1.4558091  1.41498372 1.3733599  1.33488048 1.30337782 1.23968482
 1.19163233 1.05036162 1.01326291 0.97463981 0.94672546 0.89405928
 0.81864421 0.85218356 1.00127833 1.27588247]

Speed of all vehicles (m/s): Average= 4.5, std=1.26

####################
Total number of vehicles that crossed the zero point: 137

Flow (veh/hour): 1370.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[13.24085969 13.34990819 13.3519487  13.46165859 13.56451149 13.59421745
 13.56019697 13.50378923 13.48553736 13.50984171 13.54689768 13.62498047
 13.47293558 13.41396009 

####################
Acceleration variation (m/s^2): [0.2370289711019906, 0.29426234729349426, 0.29298563763967045, 0.23610129273242494, 0.3183932478171571, 0.2733537361114134, 0.27711589182959606, 0.2915896481776529]

File: ./test_time_rollout/bcm/bcm_20240326-1215321711469732.3578181-0_emission.csv
(4, 3600)
Worst case time headway of controller(s): 
[1.27054246 1.06878362 1.03282192 1.03129511]


Average time headway of controller(s): 
[1.45520473 1.4913968  1.52989994 1.57023018]

Standard deviation of time headway of controller(s): 
[0.03490828 0.07352832 0.10600938 0.13628105]

Time headway (s): Worst = 1.03, Avg= 1.51, std= 0.09

####################
####################
Acceleration variation (m/s^2): [0.2370289711019906, 0.29426234729349426, 0.29298563763967045, 0.23610129273242494, 0.3183932478171571, 0.2733537361114134, 0.27711589182959606, 0.2915896481776529, 0.3042645462220351]

####################
####################

Final Aggregated Safety Metrics (across all files):


In [ ]:
# Data for WAR
!python classic.py --method bcm --stability --length $LENGTH --gen_emission\
--num_rollouts 5 --render --num_controlled 4


In [ ]:
!python eval_plots.py --method bcm 

__LACC__

In [ ]:
# For CAV (first get the data)
!python classic.py --method lacc --length $LENGTH --num_rollouts 5 --gen_emission\
--shock --render --num_controlled 4 --shock_start_time 11400 --shock_end_time 15000


In [22]:
# use the eval_metrics code
!python eval_metrics.py --method lacc --start_time 11400 --end_time 15000 --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/lacc/lacc_20240326-1226221711470382.5475674-0_emission.csv', './test_time_rollout/lacc/lacc_20240326-1228201711470500.8466969-0_emission.csv', './test_time_rollout/lacc/lacc_20240326-1230271711470627.0754-0_emission.csv', './test_time_rollout/lacc/lacc_20240326-1232311711470751.3487122-0_emission.csv', './test_time_rollout/lacc/lacc_20240326-1234361711470876.4382145-0_emission.csv']

Vehicle ids: ['human_0' 'human_1' 'human_2' 'human_3' 'human_4' 'human_5' 'human_6'
 'human_7' 'human_8' 'human_9' 'human_10' 'human_11' 'human_12' 'human_13'
 'human_14' 'human_15' 'human_16' 'human_17' 'lacc_0' 'lacc_1' 'lacc_2'
 'lacc_3']

eval_metrics.py:174: RuntimeWarning: divide by zero encountered in true_divide
  time_to_collision = relative_positions/relative_velocities
(4,)
Worst case time to collision for each vehicle (s): 
[-1.51690353 -1.3905344  -1.33074017 -1.09663046]

Best case time to collision for each vehicle (s): 
[-7

Total number of vehicles that crossed the zero point: 119

Flow (veh/hour): 1190.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[ 9.62664685  9.72081941  9.8136198   9.82977922  9.79173586  9.7767418
  9.70372465  9.74689566  9.73319275  9.71717654  9.66343891  9.55176309
  9.52200579  9.52899364  9.56052098  9.68710007  9.86836594 10.05837547
 10.35454094 10.24605935 10.12536253  9.86878842]

Miles per gallon (mpg): Average= 9.8, std = 0.22

####################
(22, 3600)

Average speed of each vehicle (m/s): 
[3.88768472 3.90849105 3.91009521 3.90057326 3.88418993 3.86142798
 3.8352537  3.80648983 3.77945764 3.75562699 3.73302399 3.71444531
 3.70225891 3.69792388 3.70213303 3.71406365 3.73168246 3.75309127
 3.77717736 3.80654329 3.83746225 3.86741662]

Standard deviation of speed of each vehicle (m/s): 
[3.32345603 3.33246907 3.30901675 3.28836953 3.34218366 3.32689953
 3.30330217 3.28975943 3.26028437 3.20127887 3.16461386 3.14024458
 3.0964451  3.0710

In [ ]:
# Unstable so no WAR

__Wu et. al. (20%)__

In [ ]:
# Agents are supposed to be dispersed.
# Stabalization time looks high, shocks can start later
!python test_rllib.py ./Wu_et_al/Trained_policies/20_percent/PPO_MultiAgentWaveAttenuationPOEnv-v1_436e49a2_2024-02-12_13-33-49wdl9scs5 174\
--method wu --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 4


In [23]:
# For CAV
!python eval_metrics.py --method wu --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/wu/multiagent_ring_20240326-1321161711473676.9486701-0_emission.csv', './test_time_rollout/wu/multiagent_ring_20240326-1321361711473696.58025-0_emission.csv', './test_time_rollout/wu/multiagent_ring_20240326-1324181711473858.2405589-0_emission.csv', './test_time_rollout/wu/multiagent_ring_20240326-1324371711473877.4474893-0_emission.csv', './test_time_rollout/wu/multiagent_ring_20240326-1327271711474047.0545342-0_emission.csv', './test_time_rollout/wu/multiagent_ring_20240326-1327471711474067.6080842-0_emission.csv', './test_time_rollout/wu/multiagent_ring_20240326-1330351711474235.8689075-0_emission.csv', './test_time_rollout/wu/multiagent_ring_20240326-1330561711474256.954445-0_emission.csv', './test_time_rollout/wu/multiagent_ring_20240326-1333441711474424.6597447-0_emission.csv', './test_time_rollout/wu/multiagent_ring_20240326-1334041711474444.105751-0_emission.csv']

Vehicle ids: ['rl_0_0' 'human_0_0' 'human_0_1'

(22, 3600)

Average speed of each vehicle (m/s): 
[2.79991044 2.85189335 2.85095225 2.85029058 2.84958967 2.84903244
 2.85531647 2.85469053 2.85403632 2.85345882 2.85263181 2.85166747
 2.82792204 2.82612167 2.82426016 2.82251157 2.820491   2.80622324
 2.80487681 2.80372416 2.80258951 2.80114436]

Standard deviation of speed of each vehicle (m/s): 
[0.59179183 0.56038727 0.52025006 0.52092351 0.50842663 0.50808665
 0.42537129 0.42070734 0.41793985 0.41559756 0.41404871 0.41253573
 0.43788765 0.43097905 0.42776116 0.42657219 0.41981388 0.79533233
 0.77884006 0.70487372 0.6911981  0.61548426]

Speed of all vehicles (m/s): Average= 2.83, std=0.52

####################
Total number of vehicles that crossed the zero point: 86

Flow (veh/hour): 860.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[5.37668471 5.79839022 5.77216644 5.76740326 5.6685017  5.42117994
 5.79500397 5.74004894 5.61798846 5.5873555  5.55314481 5.32058013
 5.69324613 5.64024397 5.66019613 5.6

Total number of vehicles that crossed the zero point: 77

Flow (veh/hour): 770.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[6.78491925 7.4235621  7.33235936 7.28683146 7.03340847 6.85777137
 7.41952977 7.40166735 7.40484289 7.2608697  7.08644322 6.83389689
 7.23562878 7.13169416 7.06299569 6.95415394 6.77899755 7.0500133
 7.0046139  6.99987716 7.01318155 6.96281373]

Miles per gallon (mpg): Average= 7.11, std = 0.21

####################
(22, 3600)

Average speed of each vehicle (m/s): 
[2.8618609  2.90298935 2.90249662 2.9023059  2.90186896 2.90139842
 2.8952901  2.89404967 2.89300389 2.89188029 2.89070201 2.88939072
 2.85499801 2.85373286 2.85252895 2.85137214 2.85029825 2.8647136
 2.86402975 2.86330194 2.86251584 2.86204611]

Standard deviation of speed of each vehicle (m/s): 
[0.45832614 0.76621154 0.75947066 0.74945313 0.66910551 0.66769736
 0.8047835  0.79102394 0.76987412 0.7483257  0.66656762 0.59945363
 0.71126738 0.58767153 0.57954676 0.508935

####################
Acceleration variation (m/s^2): [0.07711301088831302, 0.14295815163595768, 0.08281249042363653, 0.12455638310767604, 0.09131141966863421, 0.0709716731511715, 0.08823072584280546, 0.09989002027073704, 0.08124259149263922]

####################
####################

Final Aggregated Safety Metrics (across all files):

Time to Collision across rollouts (s):
	Worst= [-2.5313850624252825, -1.8661793599905923, -2.4663448870453153, -1.8845096585364012, -2.4712631447623243, -2.202756890564893, -2.490279796297458, -2.1610974695703447, -2.5528800266805622, -1.7362721359940976] 
	Avg= -2.24, std= 0.3

	Best: Avg= -239435.75,         std= 228952.04
	Average: Avg= -470.87, std= 226.32
	Std: Avg= 4178.38, std= 3108.38

DRAC across rollouts (m/s^2):
	Worst= [1.0032279133641406, 1.478355053252185, 1.1020604384609634, 1.531745661742373, 1.1154545281422117, 1.3939796393902424, 1.0630468671330568, 1.3130071578068163, 1.001427139897313, 1.7028722998791082] 
	Avg= 1.27, std= 0.24

CAV 

In [ ]:
# Data for WAR (Take the Safety + Stability Policy)
!python test_rllib.py ./Wu_et_al/Trained_policies/20_percent/PPO_MultiAgentWaveAttenuationPOEnv-v1_436e49a2_2024-02-12_13-33-49wdl9scs5 174\
--method wu --shock --stability --length $LENGTH --gen_emission --num_rollouts 1 --render --num_controlled 4


In [ ]:
!python eval_plots.py --method wu

__Ours (20%)__

- The local zone of __leader__ is set to 50m at training. It is kept at 50m here as well.

In [ ]:
#### Safety + Stability ####
# For CAV (first get the data). Copy the exp_configs to a folder where test_rllib.py is located
# Make sure efficiency is set to False in controller
# Make sure the estimate is turned ON in multiagent env (in 2 places)
!python test_rllib.py ./Ours/Trained_policies/20_percent/PPO_MultiAgentDensityAwareRLEnv-v1_afca7452_2024-02-11_14-35-18pt1771a1 140\
--method ours4x --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 4


In [25]:
# Since !human is present in the code, this works seamlessly
!python eval_metrics.py --method ours4x --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/ours4x/density_aware_multiagent_ring_20240327-1128091711553289.7132869-0_emission.csv', './test_time_rollout/ours4x/density_aware_multiagent_ring_20240327-1128251711553305.9182677-0_emission.csv', './test_time_rollout/ours4x/density_aware_multiagent_ring_20240327-1131491711553509.4519954-0_emission.csv', './test_time_rollout/ours4x/density_aware_multiagent_ring_20240327-1132071711553527.357204-0_emission.csv', './test_time_rollout/ours4x/density_aware_multiagent_ring_20240327-1135531711553753.167958-0_emission.csv', './test_time_rollout/ours4x/density_aware_multiagent_ring_20240327-1136111711553771.9257028-0_emission.csv', './test_time_rollout/ours4x/density_aware_multiagent_ring_20240327-1140151711554015.7593565-0_emission.csv', './test_time_rollout/ours4x/density_aware_multiagent_ring_20240327-1140361711554036.4260347-0_emission.csv', './test_time_rollout/ours4x/density_aware_multiagent_ring_20240327-1144241711554264

(4,)
Worst case drac for each vehicle (m/s^2): 
[0.04517942 0.04287567 0.00894501 0.33891014]

DRAC (m/s^2):
	Worst= 0.34

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[ 8.80692833  8.84045927  8.96002776  8.38142625 10.35307143 10.26282073
 10.36584691 10.26186853 10.29770012 10.19811402 10.15248892 10.02833907
  9.86528954  9.76035529  9.48496231  9.38251125  9.3801544   9.24500782
  9.05819667  8.92281337  8.54007319  8.56158189]

Miles per gallon (mpg): Average= 9.51, std = 0.66

####################
(22, 3600)

Average speed of each vehicle (m/s): 
[3.49001541 3.49107049 3.49303157 3.49760559 3.48415924 3.48438679
 3.48441946 3.48436586 3.48440229 3.48445629 3.48468238 3.48515915
 3.48572774 3.48624488 3.48710615 3.48794238 3.48847148 3.48869129
 3.4888996  3.48882567 3.48902673 3.48952703]

Standard deviation of speed of each vehicle (m/s): 
[0.09562054 0.10092513 0.10122408 0.11643965 1.16773725 1.14101816
 1.11876289 1.09546585 1.0777363  1.06381747

Total number of vehicles that crossed the zero point: 112

Flow (veh/hour): 1120.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[ 9.54043729  9.80012499 10.05583262  8.30018191 10.38378461 10.25459214
 10.24947175 10.18325438 10.08941679  9.92517325  9.89046347  9.72149489
  9.65555035  9.38762784  9.26799269  9.13499781  9.03969705  9.03248888
  8.80794749  8.83071938  8.66928628  8.67471991]

Miles per gallon (mpg): Average= 9.5, std = 0.6

####################
(22, 3600)

Average speed of each vehicle (m/s): 
[3.45398496 3.45414699 3.45419511 3.45456028 3.44756386 3.44758815
 3.44801038 3.4487872  3.44958895 3.45034253 3.45100658 3.45135823
 3.4521765  3.4525689  3.45281817 3.45305539 3.4533546  3.45367055
 3.45373428 3.45399339 3.4539956  3.45397686]

Standard deviation of speed of each vehicle (m/s): 
[0.27108486 0.27388752 0.26962955 0.261376   1.38179684 1.33877718
 1.30412252 1.26876435 1.15636894 1.11588442 1.07921587 1.01786289
 0.97584566 0.9300

####################
Acceleration variation (m/s^2): [0.2336136836899271, 0.35308471615842113, 0.4039870147675562, 0.3721381330837912, 0.4181056560008979, 0.37484560950808304, 0.2911519319051073]

File: ./test_time_rollout/ours4x/density_aware_multiagent_ring_20240327-1144241711554264.3853307-0_emission.csv
(4, 3600)
Worst case time headway of controller(s): 
[1.02337307 1.0288796  1.02574334 3.67977107]


Average time headway of controller(s): 
[1.10411051 1.115632   1.35457291 8.30014043]

Standard deviation of time headway of controller(s): 
[0.10418228 0.08283767 0.42509112 2.4754519 ]

Time headway (s): Worst = 1.02, Avg= 2.97, std= 0.77

####################
####################
Acceleration variation (m/s^2): [0.2336136836899271, 0.35308471615842113, 0.4039870147675562, 0.3721381330837912, 0.4181056560008979, 0.37484560950808304, 0.2911519319051073, 0.37519000370205424]

File: ./test_time_rollout/ours4x/density_aware_multiagent_ring_20240327-1144481711554288.7257478-0_emission.c

In [ ]:
#### Efficiency ####
# Make sure to set the efficiency to true at test time in the controller file and have those files at exp_configs
# Make sure to have the right leader
# Set efficiency to True in TrainedAgentController

!python test_rllib.py ./Ours/Trained_policies/20_percent/PPO_MultiAgentDensityAwareRLEnv-v1_33361ae0_2024-02-11_20-29-43f1pzfgrz 140\
--method ours4x --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 4


In [ ]:
!python eval_metrics.py --method ours4x --save_plots

In [ ]:
# Data for WAR (taken on the Safety + Stability model). Make sure everything is tuned for Safety + Stability
!python test_rllib.py ./Ours/Trained_policies/20_percent/PPO_MultiAgentDensityAwareRLEnv-v1_afca7452_2024-02-11_14-35-18pt1771a1 140\
--method ours4x --shock --stability --length $LENGTH --gen_emission --num_rollouts 1 --render --num_controlled 4


In [ ]:
!python eval_plots.py --method ours4x

### 4. (40%) Multiple Vehicle Control

- 9 vehicles out of 22

In [ ]:
# set parameters here
NUM_ROLLOUTS = 5
LENGTH = 260
NUM_CONTROLLED = 9

### FS

In [ ]:
!python classic.py --method fs --length $LENGTH\
--gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render --num_controlled $NUM_CONTROLLED

In [26]:
# use the eval_metrics code
!python eval_metrics.py --method fs --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/fs/fs_20240326-1556351711482995.3867946-0_emission.csv', './test_time_rollout/fs/fs_20240326-1558381711483118.3282375-0_emission.csv', './test_time_rollout/fs/fs_20240326-1600441711483244.1012588-0_emission.csv', './test_time_rollout/fs/fs_20240326-1602371711483357.1959002-0_emission.csv', './test_time_rollout/fs/fs_20240326-1604451711483485.3611019-0_emission.csv', './test_time_rollout/fs/fs_20240326-1606531711483613.1846836-0_emission.csv', './test_time_rollout/fs/fs_20240326-1608551711483735.981296-0_emission.csv', './test_time_rollout/fs/fs_20240326-1611031711483863.9627314-0_emission.csv', './test_time_rollout/fs/fs_20240326-1613121711483992.2300882-0_emission.csv', './test_time_rollout/fs/fs_20240326-1615191711484119.5354905-0_emission.csv']

Vehicle ids: ['human_0' 'human_1' 'human_2' 'human_3' 'human_4' 'human_5' 'human_6'
 'human_7' 'human_8' 'human_9' 'human_10' 'human_11' 'human_12' 'fs_0'
 'fs_1' 'fs_2' 'fs

(9,)
Worst case drac for each vehicle (m/s^2): 
[0.02161869 0.01841855 0.01056172 0.00442276 0.00328797 0.00330662
 0.00363376 0.00368615 0.65548339]

DRAC (m/s^2):
	Worst= 0.66

(9,)
Worst case time to collision for each vehicle (s): 
[-13.3231516  -14.98888172 -15.72350219 -17.95796228 -20.24480694
 -19.47225646 -26.92339656 -38.52206321  -3.2917086 ]

Best case time to collision for each vehicle (s): 
[-6.91952857e+15 -6.67251336e+15 -6.60216178e+15 -6.17431747e+15
 -6.28420767e+15 -3.20540103e+15 -3.04513684e+15 -8.58862788e+14
 -4.75938902e+04]

Average time to collision for each vehicle (s): 
[-4.30715518e+15 -4.29320087e+15 -4.52144146e+15 -4.83135537e+15
 -5.18447080e+15 -2.97519567e+15 -8.49888026e+13 -2.41708206e+13
 -4.97244700e+02]

Standard deviation of time to collision for each vehicle (s): 
[2.69897099e+15 2.67211805e+15 2.37037912e+15 2.14070353e+15
 1.84916384e+15 8.24359266e+14 4.55994806e+14 1.28450353e+14
 2.15624188e+03]

Time to Collision (s):
	Worst= -3.29
	Best

(22, 3600)

Miles per gallon for each vehicle (mpg):
[12.01858961 11.99199254 11.90161293 11.94355563 11.86942166 11.80521503
 11.77285391 11.71421721 11.6763487  11.62457054 11.54771976 11.55513992
 11.46054524 11.22895776 11.21941097 11.22119002 11.22085094 11.22442157
 11.22432964 11.22119728 11.22376805 11.22268039]

Miles per gallon (mpg): Average= 11.54, std = 0.3

####################
(22, 3600)

Average speed of each vehicle (m/s): 
[4.50421278 4.50440636 4.5042317  4.50438601 4.50457102 4.50454875
 4.5044724  4.50458595 4.50470275 4.50507511 4.50496651 4.50453818
 4.50446272 4.50441509 4.50441268 4.50439208 4.50438677 4.50438409
 4.50438281 4.50438225 4.50438228 4.50438104]

Standard deviation of speed of each vehicle (m/s): 
[0.75677362 0.74462353 0.73146066 0.72160582 0.7084887  0.69637101
 0.68685235 0.67483407 0.65970438 0.64915238 0.61156059 0.58655881
 0.55658823 0.31141188 0.30751407 0.30463726 0.30212582 0.2999532
 0.29803153 0.29641552 0.29511218 0.2941362 ]

Speed of

####################
Acceleration variation (m/s^2): [0.2781330422047812]

File: ./test_time_rollout/fs/fs_20240326-1600441711483244.1012588-0_emission.csv
(9, 3600)
Worst case time headway of controller(s): 
[1.1623072  1.1622217  1.16088182 1.16228677 1.1622101  1.16219577
 1.16264465 1.16740707 1.17318328]


Average time headway of controller(s): 
[1.20941642 1.20747621 1.20683627 1.20686212 1.20769686 1.20894882
 1.21005524 1.21189577 4.29301827]

Standard deviation of time headway of controller(s): 
[0.17127914 0.15428127 0.1457251  0.14081724 0.13867815 0.13840165
 0.13915711 0.14118463 1.19062954]

Time headway (s): Worst = 1.16, Avg= 1.55, std= 0.26

####################
####################
Acceleration variation (m/s^2): [0.2781330422047812, 0.4368101669662806]

File: ./test_time_rollout/fs/fs_20240326-1602371711483357.1959002-0_emission.csv
(9, 3600)
Worst case time headway of controller(s): 
[1.16897693 1.16560439 1.16560249 1.16551446 1.16549645 1.165884
 1.16593941 1.1700

In [ ]:
# Data for WAR Desired velocity should be set to 4.82 here
!python classic.py --method fs --stability --length $LENGTH\
--num_rollouts 5 --gen_emission --render --num_controlled $NUM_CONTROLLED


In [ ]:
# WAR
!python eval_plots.py --method fs 

### PIwS

In [ ]:
!python classic.py --method piws --length $LENGTH\
--gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render --num_controlled $NUM_CONTROLLED


In [27]:
!python eval_metrics.py --method pi --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/pi/piws_20240326-1410201711476620.2579749-0_emission.csv', './test_time_rollout/pi/piws_20240326-1412241711476744.1885464-0_emission.csv', './test_time_rollout/pi/piws_20240326-1414421711476882.5038776-0_emission.csv', './test_time_rollout/pi/piws_20240326-1417131711477033.6181107-0_emission.csv', './test_time_rollout/pi/piws_20240326-1419521711477192.1469548-0_emission.csv', './test_time_rollout/pi/piws_20240326-1439451711478385.3533561-0_emission.csv', './test_time_rollout/pi/piws_20240326-1441541711478514.5120745-0_emission.csv', './test_time_rollout/pi/piws_20240326-1444141711478654.1735861-0_emission.csv', './test_time_rollout/pi/piws_20240326-1446101711478770.087185-0_emission.csv', './test_time_rollout/pi/piws_20240326-1448201711478900.3526344-0_emission.csv']

Vehicle ids: ['human_0' 'human_1' 'human_2' 'human_3' 'human_4' 'human_5' 'human_6'
 'human_7' 'human_8' 'human_9' 'human_10' 'human_11' 'human_12' 'piws

(9,)
Worst case drac for each vehicle (m/s^2): 
[0.00669893 0.06626089 0.65189189 1.7708391  1.63327399 1.09709623
 0.49801099 1.02764523 1.79814566]

DRAC (m/s^2):
	Worst= 1.8

(9,)
Worst case time to collision for each vehicle (s): 
[-10.11758124  -5.60946642  -8.6126321  -14.92713432  -9.48438834
  -2.38544987  -1.72138785  -1.7010746   -4.18608002]

Best case time to collision for each vehicle (s): 
[ -130159.50196712 -1372550.83501437  -504080.96978485   -70859.01938853
  -290481.33235189   -65125.32603793   -61852.8205541    -33031.94084907
   -31151.66173404]

Average time to collision for each vehicle (s): 
[-2183.47289473 -6287.29713266 -1329.77858066  -621.71946918
 -1214.10559742  -380.17316899  -499.54923719  -242.36696035
  -112.52254166]

Standard deviation of time to collision for each vehicle (s): 
[ 8646.99279706 64392.16520055 15958.60507369  2909.78898666
  9599.26427587  2157.68636832  2849.52776177  1391.89462812
   887.91219139]

Time to Collision (s):
	Worst= -1.

(22, 3600)

Average speed of each vehicle (m/s): 
[4.46715082 4.46432311 4.46157365 4.45921438 4.45696458 4.45479935
 4.45224154 4.44988172 4.44767092 4.44532375 4.44321586 4.4411241
 4.43938273 4.43815437 4.4369009  4.4362453  4.43593279 4.43590643
 4.4359161  4.44502626 4.47414736 4.48890405]

Standard deviation of speed of each vehicle (m/s): 
[0.79454391 0.77254438 0.74949941 0.71606724 0.69606034 0.67406561
 0.65241983 0.63190529 0.58279487 0.54111891 0.51972568 0.24530402
 0.24250059 0.15530133 0.19565771 0.23501948 0.26973042 0.32561165
 0.33720112 0.34125081 0.38082363 0.48186175]

Speed of all vehicles (m/s): Average= 4.45, std=0.48

####################
Total number of vehicles that crossed the zero point: 135

Flow (veh/hour): 1350.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[11.20950728 11.18690323 11.07096824 10.76406874 10.70318342 10.61304548
 10.63357672 10.41579573 10.19708122 10.07529224 10.054948    9.97640805
  9.95809539  9.88869956

(9, 3600)
Worst case time headway of controller(s): 
[1.02074122 1.02080454 1.02116708 1.02113264 1.01637898 1.02092084
 1.01981279 1.01884471 1.01744129]


Average time headway of controller(s): 
[1.38509668 1.41994772 1.46262436 1.51276762 1.60383614 1.70279234
 1.65406151 1.55770706 1.65969328]

Standard deviation of time headway of controller(s): 
[0.62368919 0.65349621 0.6860118  0.68262466 0.71907215 0.74547449
 0.71601436 0.69134898 0.78183641]

Time headway (s): Worst = 1.02, Avg= 1.55, std= 0.7

####################
####################
Acceleration variation (m/s^2): [0.14712865978453493, 0.2344122077677847]

File: ./test_time_rollout/pi/piws_20240326-1417131711477033.6181107-0_emission.csv
(9, 3600)
Worst case time headway of controller(s): 
[1.0229153  1.03521752 1.02409546 1.02326861 1.01724263 1.01964286
 1.02025903 1.01862978 1.01797888]


Average time headway of controller(s): 
[1.44553341 1.53770521 1.62471761 1.76694315 1.89083294 2.05742944
 2.14386341 2.086771   2.0

In [ ]:
# Data for WAR
!python classic.py --method piws --stability --length $LENGTH --num_rollouts 5 --gen_emission\
--render --num_controlled $NUM_CONTROLLED


In [ ]:
# WAR
!python eval_plots.py --method pi 

### BCM

In [ ]:
# For CAV (first get the data)
!python classic.py --method bcm --length $LENGTH\
--num_rollouts $NUM_ROLLOUTS --gen_emission --shock --num_controlled $NUM_CONTROLLED


In [28]:
# use the eval_metrics code
!python eval_metrics.py --method bcm --save_plots


Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/bcm/bcm_20240326-1422281711477348.626685-0_emission.csv', './test_time_rollout/bcm/bcm_20240326-1424261711477466.7236097-0_emission.csv', './test_time_rollout/bcm/bcm_20240326-1426251711477585.3272057-0_emission.csv', './test_time_rollout/bcm/bcm_20240326-1428231711477703.5138621-0_emission.csv', './test_time_rollout/bcm/bcm_20240326-1430221711477822.6833725-0_emission.csv', './test_time_rollout/bcm/bcm_20240326-1444201711478660.0371747-0_emission.csv', './test_time_rollout/bcm/bcm_20240326-1445491711478749.3814805-0_emission.csv', './test_time_rollout/bcm/bcm_20240326-1447221711478842.2594264-0_emission.csv', './test_time_rollout/bcm/bcm_20240326-1449101711478950.8709342-0_emission.csv', './test_time_rollout/bcm/bcm_20240326-1450571711479057.227868-0_emission.csv']

Vehicle ids: ['human_0' 'human_1' 'human_2' 'human_3' 'human_4' 'human_5' 'human_6'
 'human_7' 'human_8' 'human_9' 'human_10' 'human_11' 'human_12' 'bcm_0

(9,)
Worst case drac for each vehicle (m/s^2): 
[5.83066032e-04 1.20419751e-03 2.49154679e-03 5.33406444e-03
 1.20945347e-02 2.86454330e-02 7.39682968e-02 2.47220702e-01
 1.32835628e+00]

DRAC (m/s^2):
	Worst= 1.33

(9,)
Worst case time to collision for each vehicle (s): 
[-3.45045364 -3.00600475 -2.97543063 -3.05784417 -3.19317125 -3.16175761
 -2.82821924 -2.41753834 -2.15580913]

Best case time to collision for each vehicle (s): 
[ -491591.64953652  -842636.70808337  -903316.2549932   -191223.84234315
 -1012067.60412206  -286896.40365059  -233108.63544545   -47676.65482857
  -355848.2338187 ]

Average time to collision for each vehicle (s): 
[-2644.27097534 -2520.2448363  -2958.0277874  -1551.9483589
 -1888.77942191  -732.20391937  -576.3363765   -396.29495846
  -394.66160363]

Standard deviation of time to collision for each vehicle (s): 
[18814.27600943 27360.97558541 30013.25087294  7225.9749612
 27991.33747734  7182.52221232  6145.27021008  2116.20808033
  8324.96304079]

Time to

(22, 3600)

Average speed of each vehicle (m/s): 
[4.59722953 4.59754242 4.59772333 4.59732529 4.59664768 4.5965491
 4.59673005 4.59688696 4.596947   4.59705067 4.59711707 4.59714826
 4.59726894 4.59718147 4.59714283 4.59712073 4.59710124 4.59707776
 4.59704946 4.59702235 4.59701272 4.59705598]

Standard deviation of speed of each vehicle (m/s): 
[0.76755424 0.75252021 0.74154866 0.72785106 0.72102276 0.71911612
 0.67997298 0.67964405 0.66450402 0.62896288 0.40639161 0.40484829
 0.15328738 0.1508925  0.1613039  0.19332357 0.2339891  0.27986565
 0.3316571  0.39332115 0.47357998 0.58878984]

Speed of all vehicles (m/s): Average= 4.6, std=0.49

####################
Total number of vehicles that crossed the zero point: 140

Flow (veh/hour): 1400.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[12.11763158 12.0810778  11.95320516 11.91446403 11.88096844 11.77339889
 11.78483234 11.75476016 11.70719691 11.66937058 11.58841286 11.62073994
 11.60083539 11.50446083 

(9, 3600)
Worst case time headway of controller(s): 
[1.30468587 1.38534821 1.43774957 1.47680909 1.50564619 1.51918967
 1.52609675 1.52604258 1.51567889]


Average time headway of controller(s): 
[1.44813522 1.46887969 1.48963148 1.51037274 1.53109735 1.55180337
 1.57249072 1.59316127 1.61382362]

Standard deviation of time headway of controller(s): 
[0.01481457 0.01068451 0.00850439 0.00763232 0.00776751 0.0088837
 0.01119287 0.01511845 0.02139064]

Time headway (s): Worst = 1.3, Avg= 1.53, std= 0.01

####################
####################
Acceleration variation (m/s^2): [0.1990289601178742, 0.2312626667161847]

File: ./test_time_rollout/bcm/bcm_20240326-1428231711477703.5138621-0_emission.csv
(9, 3600)
Worst case time headway of controller(s): 
[1.25704816 1.3520483  1.34076641 1.30041    1.24310439 1.15777168
 1.0453711  1.03772442 1.03642139]


Average time headway of controller(s): 
[1.4464489  1.46473945 1.48308748 1.50145309 1.51978262 1.53797864
 1.55595218 1.57539805 1.596

In [ ]:
# Data for WAR
!python classic.py --method bcm --stability --length $LENGTH --gen_emission\
--num_rollouts 5 --render --num_controlled $NUM_CONTROLLED


In [ ]:
# WAR
!python eval_plots.py --method bcm 

### LACC

In [ ]:
# For CAV (first get the data)
!python classic.py --method lacc --length $LENGTH\
--num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled $NUM_CONTROLLED --shock_start_time 11400 --shock_end_time 15000


In [29]:
# use the eval_metrics code
!python eval_metrics.py --method lacc --start_time 11400 --end_time 15000 --save_plots


Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/lacc/lacc_20240326-1423071711477387.0918489-0_emission.csv', './test_time_rollout/lacc/lacc_20240326-1425381711477538.2298028-0_emission.csv', './test_time_rollout/lacc/lacc_20240326-1428091711477689.944365-0_emission.csv', './test_time_rollout/lacc/lacc_20240326-1430431711477843.8793879-0_emission.csv', './test_time_rollout/lacc/lacc_20240326-1433151711477995.8284006-0_emission.csv', './test_time_rollout/lacc/lacc_20240326-1451311711479091.4056506-0_emission.csv', './test_time_rollout/lacc/lacc_20240326-1454001711479240.0356288-0_emission.csv', './test_time_rollout/lacc/lacc_20240326-1456281711479388.2425249-0_emission.csv', './test_time_rollout/lacc/lacc_20240326-1458571711479537.6987135-0_emission.csv', './test_time_rollout/lacc/lacc_20240326-1501261711479686.6767118-0_emission.csv']

Vehicle ids: ['human_0' 'human_1' 'human_2' 'human_3' 'human_4' 'human_5' 'human_6'
 'human_7' 'human_8' 'human_9' 'human_10' 'human_

(9,)
Worst case drac for each vehicle (m/s^2): 
[0.09290601 0.10750282 0.12534734 0.14752825 0.17558121 0.2121187
 0.26217346 0.33828731 0.56889835]

DRAC (m/s^2):
	Worst= 0.57

(9,)
Worst case time to collision for each vehicle (s): 
[-6.95341461 -6.46275471 -5.98614056 -5.52215528 -5.06964996 -4.62774922
 -4.19000185 -3.67147908 -2.9267702 ]

Best case time to collision for each vehicle (s): 
[-277484.74227363 -307033.74859853 -155461.50969581 -153851.96974355
 -165031.17827871  -25976.67637662  -86577.17399276  -19210.54324203
  -76490.42703912]

Average time to collision for each vehicle (s): 
[-310.1574817  -330.22561885 -305.00411403 -247.70057969 -398.35263264
 -145.68443869 -193.79387465 -136.85257234 -147.74327375]

Standard deviation of time to collision for each vehicle (s): 
[6571.74513875 7303.86641207 4462.27866628 3804.35772497 5858.52029305
  948.29499166 2477.32300348  701.56477938 1939.50683129]

Time to Collision (s):
	Worst= -2.93
	Best= -307033.75
	Average= -246.17

(22, 3600)

Average speed of each vehicle (m/s): 
[4.65726593 4.65455276 4.648322   4.63962133 4.63003055 4.62041019
 4.61104808 4.60277434 4.59591067 4.59068974 4.58722547 4.58582724
 4.58630821 4.58843065 4.59015903 4.59457375 4.60163734 4.61108888
 4.62232596 4.63427806 4.64536296 4.65367626]

Standard deviation of speed of each vehicle (m/s): 
[1.90496004 1.79068198 1.77847051 1.74235454 1.71774749 1.68427107
 1.64819999 1.60960822 1.56589191 1.48929464 1.45179561 1.43116903
 1.37138834 1.20837102 1.26293086 1.32314627 1.38994413 1.46366917
 1.54378835 1.62887679 1.71745565 1.80852384]

Speed of all vehicles (m/s): Average= 4.62, std=1.57

####################
Total number of vehicles that crossed the zero point: 140

Flow (veh/hour): 1400.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[14.55241417 14.63938952 14.70906731 14.6386311  14.62245447 14.39182235
 14.46227545 14.69419378 14.87347622 14.77772607 14.92231529 14.67142438
 14.49876991 14.5320094

Acceleration variation (m/s^2): [0.2012758023207635]

File: ./test_time_rollout/lacc/lacc_20240326-1428091711477689.944365-0_emission.csv
(9, 3600)
Worst case time headway of controller(s): 
[1.39629461 1.39606763 1.39582417 1.39556247 1.39528022 1.39497428
 1.3946461  1.39323684 1.38832783]


Average time headway of controller(s): 
[1.40006246 1.40007196 1.40008125 1.40009013 1.4000985  1.40010627
 1.40011328 1.40011911 1.40012402]

Standard deviation of time headway of controller(s): 
[0.00201295 0.00212337 0.002241   0.00236674 0.00250217 0.00264988
 0.0028147  0.00300822 0.00327465]

Time headway (s): Worst = 1.39, Avg= 1.4, std= 0.0

####################
####################
Acceleration variation (m/s^2): [0.2012758023207635, 0.5051131178899649]

File: ./test_time_rollout/lacc/lacc_20240326-1430431711477843.8793879-0_emission.csv
(9, 3600)
Worst case time headway of controller(s): 
[1.39027704 1.38941037 1.38841914 1.38726943 1.38593122 1.38438094
 1.38256327 1.37735379 1.1490216

In [ ]:
# Data for WAR
!python classic.py --method lacc --stability --length $LENGTH --num_rollouts 5 --gen_emission\
--render --num_controlled $NUM_CONTROLLED --shock_start_time 11400 --shock_end_time 15000

In [ ]:
# WAR
!python eval_plots.py --method lacc --start_time 11400 --end_time 15000 --propogate_time 200

### Wu et. al. (40%)

In [ ]:
# Agents are supposed to be dispersed.
# Stabalization time looks high, shocks can start later
!python test_rllib.py ./Wu_et_al/Trained_policies/40_percent/PPO_MultiAgentWaveAttenuationPOEnv-v1_b8f3c414_2024-02-14_13-13-15_l2ifa89 92\
--method wu --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 9


In [30]:
# For CAV
!python eval_metrics.py --method wu --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/wu/multiagent_ring_20240327-1209141711555754.09733-0_emission.csv', './test_time_rollout/wu/multiagent_ring_20240327-1209191711555759.6766565-0_emission.csv', './test_time_rollout/wu/multiagent_ring_20240327-1213561711556036.1323743-0_emission.csv', './test_time_rollout/wu/multiagent_ring_20240327-1213581711556038.7502773-0_emission.csv', './test_time_rollout/wu/multiagent_ring_20240327-1218231711556303.5496683-0_emission.csv', './test_time_rollout/wu/multiagent_ring_20240327-1218251711556305.06416-0_emission.csv', './test_time_rollout/wu/multiagent_ring_20240327-1222251711556545.4020603-0_emission.csv', './test_time_rollout/wu/multiagent_ring_20240327-1222261711556546.4138875-0_emission.csv', './test_time_rollout/wu/multiagent_ring_20240327-1227021711556822.280683-0_emission.csv', './test_time_rollout/wu/multiagent_ring_20240327-1227061711556826.736833-0_emission.csv']

Vehicle ids: ['rl_0_0' 'human_0_0' 'rl_1_0' 'hum

(9,)
Worst case drac for each vehicle (m/s^2): 
[0.44944141 0.60312157 0.7152197  0.73535029 1.45520596 0.21464105
 0.81272241 0.13523248 0.33239203]

DRAC (m/s^2):
	Worst= 1.46

(9,)
Worst case time to collision for each vehicle (s): 
[-4.34571096 -3.6632187  -2.74477235 -4.67979877 -5.09836729 -4.99213973
 -5.72176064 -4.95816051 -5.5370027 ]

Best case time to collision for each vehicle (s): 
[-257309.72131095   -6957.00218264  -42597.78466262   -9336.63236958
   -8738.30335772  -77204.5723085  -280941.85390947 -437846.80870887
 -106173.58208079]

Average time to collision for each vehicle (s): 
[ -940.19068811   -56.65668821  -306.61022316   -90.85083806
   -81.31328201  -638.8493096  -1374.02268664  -917.16161448
  -667.15336412]

Standard deviation of time to collision for each vehicle (s): 
[ 8802.75106172   276.59255383  2063.50510951   475.4367265
   467.04880994  3279.83990381 11692.46585082 14381.26143772
  4960.78196061]

Time to Collision (s):
	Worst= -2.74
	Best= -437846.

(22, 3600)

Average speed of each vehicle (m/s): 
[2.46963466 2.55991048 2.55639767 2.54409031 2.53935337 2.53453385
 2.52268305 2.51984222 2.51741377 2.51442216 2.51130372 2.50783522
 2.50456035 2.5025052  2.49928904 2.49602208 2.49310816 2.48980454
 2.4876234  2.48448566 2.48116743 2.4732806 ]

Standard deviation of speed of each vehicle (m/s): 
[0.61652176 0.67864566 0.66347558 0.60572614 0.59701046 0.58758493
 0.53462576 0.52940768 0.50367036 0.49851546 0.49225849 0.46395357
 0.45650091 0.41383087 0.40553387 0.39747709 0.38059072 0.37258394
 0.72903901 0.67556097 0.66914028 0.62465462]

Speed of all vehicles (m/s): Average= 2.51, std=0.54

####################
Total number of vehicles that crossed the zero point: 76

Flow (veh/hour): 760.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[0.80186282 0.83066623 0.81075923 1.00057831 0.97780537 0.95789697
 0.93814128 0.91628924 1.27990278 1.26246243 1.27335116 1.11431539
 1.07548562 1.04273008 1.01525574 0.9

(9, 3600)
Worst case time headway of controller(s): 
[1.60309268 1.61343153 1.56315361 1.56578608 1.61300835 1.60368223
 1.60788523 1.61243946 1.58500526]


Average time headway of controller(s): 
[2.62146403 4.1848193  2.26399594 4.71415571 5.08502089 5.56760447
 3.49255439 2.86346205 3.15441313]

Standard deviation of time headway of controller(s): 
[0.90460276 2.42278851 0.36799606 2.4363088  4.99512267 4.66499223
 1.04993622 0.89249778 1.27845531]

Time headway (s): Worst = 1.56, Avg= 3.77, std= 2.11

####################
####################
Acceleration variation (m/s^2): [0.14054984409870538, 0.16190004662643595]

File: ./test_time_rollout/wu/multiagent_ring_20240327-1213581711556038.7502773-0_emission.csv
(9, 3600)
Worst case time headway of controller(s): 
[1.63644186 1.64591703 1.59386307 1.57747766 1.57890951 1.58022266
 1.58087229 1.6493044  1.63355301]


Average time headway of controller(s): 
[ 48.86727591  68.83973827  37.977749    35.11257952  65.94658324
  15.63049997 

In [ ]:
# Data for WAR

In [ ]:
# WAR

### Ours (40%)

In [ ]:
#### Safety + Stability ####
# For CAV (first get the data). Copy the exp_configs to a folder where test_rllib.py is located
# Make sure efficiency is set to False in controller
# Make sure the estimate is turned ON in multiagent env (in 2 places)
!python test_rllib.py ./Ours/Trained_policies/40_percent/PPO_MultiAgentDensityAwareRLEnv-v1_589d08ce_2024-02-14_13-03-242t0mb06j 60\
--method ours9x --length $LENGTH --num_rollouts 20 --gen_emission --shock --render --num_controlled $NUM_CONTROLLED



In [32]:
# Since !human is present in the code, this works seamlessly
!python eval_metrics.py --method ours9x --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/ours9x/density_aware_multiagent_ring_20240327-1351531711561913.8572984-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240327-1351541711561914.870917-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240327-1351571711561917.4615736-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240327-1357031711562223.613163-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240327-1357071711562227.076018-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240327-1357091711562229.6841137-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240327-1402281711562548.3451078-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240327-1402301711562550.506555-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240327-1402301711562550.9

Standard deviation of time to collision for each vehicle (s): 
[  994.08680449  1410.51664572  4504.85870758  6350.80712304
 24000.86094642  4069.21334961  1404.48563014  1023.90660967
 13164.72493874]

Time to Collision (s):
	Worst= -2.37
	Best= -1032203.63
	Average= -401.63
	Std= 6324.83

####################
(9,)
Worst case drac for each vehicle (m/s^2): 
[0.03747761 0.03005051 0.06684671 0.02458007 0.03536008 0.04751295
 0.03896835 0.05099891 1.02786386]

DRAC (m/s^2):
	Worst= 1.03

(9,)
Worst case time to collision for each vehicle (s): 
[-11.95741566 -11.37042232 -13.68338574 -11.32881106  -9.49604165
 -11.7365469   -9.22772221 -23.6541705   -8.63004677]

Best case time to collision for each vehicle (s): 
[-120848.00042326  -35854.73101245  -23942.44144639  -27214.83688566
  -33826.32586327  -31263.46638641 -757008.59267182  -27546.5253231
  -28707.54094134]

Average time to collision for each vehicle (s): 
[-281.44513261 -223.43977374 -229.37286811 -242.10412557 -200.97892408
 -

Total number of vehicles that crossed the zero point: 125

Flow (veh/hour): 1250.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[10.97113168 10.92335236 10.84259989 10.92580942 10.95718802 10.86814784
 10.81907624 10.83399261  9.44378063 12.97094066 12.98305896 12.7947878
 12.51267836 12.16559119 11.71438567 11.5566106  11.40507995 11.25215017
 10.88227318 10.45982975 10.26534756 10.01623322]

Miles per gallon (mpg): Average= 11.25, std = 0.92

####################
(22, 3600)

Average speed of each vehicle (m/s): 
[3.89207595 3.89274177 3.89314485 3.89314868 3.89169005 3.89121427
 3.89118935 3.89145931 3.89067448 3.88986808 3.89036592 3.89076273
 3.89101439 3.89130705 3.89114049 3.89107775 3.89100006 3.89101914
 3.89122055 3.89158899 3.89176611 3.89193441]

Standard deviation of speed of each vehicle (m/s): 
[0.25032118 0.24906796 0.24667598 0.24471433 0.24342373 0.24191847
 0.23942651 0.23814988 0.22957543 1.54455165 1.43466109 1.37689036
 1.3322466  1.25

(9, 3600)
Worst case time headway of controller(s): 
[1.02038569 1.01927892 1.01921724 1.01538415 1.01942678 1.01622769
 1.02008732 1.02269295 1.03665338]


Average time headway of controller(s): 
[1.07120527 1.06546015 1.06172185 1.06110591 1.06336285 1.06520492
 1.05644333 1.05745012 8.02361804]

Standard deviation of time headway of controller(s): 
[0.03516265 0.03701344 0.02994318 0.02794343 0.03099841 0.0322566
 0.0277225  0.02554232 2.4258149 ]

Time headway (s): Worst = 1.02, Avg= 1.84, std= 0.3

####################
####################
Acceleration variation (m/s^2): [0.41090701701068477]

File: ./test_time_rollout/ours9x/density_aware_multiagent_ring_20240327-1351571711561917.4615736-0_emission.csv
(9, 3600)
Worst case time headway of controller(s): 
[1.01524211 1.01330503 1.00884284 1.0068348  1.00416533 1.00314379
 1.00103791 1.00179508 1.02925285]


Average time headway of controller(s): 
[ 1.0790987   1.07617067  1.06417906  1.06721226  1.07307367  1.07470102
  1.06684119

In [ ]:
#### Efficiency ####
# Make sure to set the efficiency to true at test time in the controller file and have those files at exp_configs
# Make sure to have the right leader
!python test_rllib.py ./Ours/Trained_policies/40_percent/PPO_MultiAgentDensityAwareRLEnv-v1_92538b0c_2024-02-15_14-36-53fn6l9uvs 140\
--method ours9x --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --render --shock --num_controlled $NUM_CONTROLLED


In [2]:
# Since !human is present in the code, this works seamlessly
!python eval_metrics.py --method ours9x --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/ours9x/density_aware_multiagent_ring_20240328-1332271711647147.9690323-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240328-1332291711647149.3188415-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240328-1337431711647463.2143638-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240328-1337431711647463.4356213-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240328-1342511711647771.5272796-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240328-1342521711647772.4146538-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240328-1347551711648075.8010864-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240328-1347571711648077.8549862-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240328-13525817116483

(9,)
Worst case drac for each vehicle (m/s^2): 
[0.0712485  0.05100663 0.06144022 0.04875027 0.0628692  0.06698184
 0.07764625 0.1121422  0.78910665]

DRAC (m/s^2):
	Worst= 0.79

(9,)
Worst case time to collision for each vehicle (s): 
[-2.04510937 -3.90059853 -3.93497257 -3.49004429 -3.54930733 -3.32856855
 -2.98594429 -2.58259132 -1.64771624]

Best case time to collision for each vehicle (s): 
[-93289.49145805 -11605.15725012 -32943.1739     -12418.05074794
 -31540.12735257 -12719.2502473  -52162.81292689 -59974.62874733
  -5707.2812551 ]

Average time to collision for each vehicle (s): 
[-212.20985368 -174.01258412 -215.38320621 -171.96574795 -242.23586529
 -190.41967746 -349.15487023 -339.41344071  -48.60920546]

Standard deviation of time to collision for each vehicle (s): 
[2468.59297267  650.34365424 1233.61993726  658.89213193 1330.34963569
  637.00243893 2007.40063692 1963.97842283  195.40460378]

Time to Collision (s):
	Worst= -1.65
	Best= -93289.49
	Average= -215.93
	Std= 12

(22, 3600)

Average speed of each vehicle (m/s): 
[4.44346512 4.44324177 4.44293657 4.44238836 4.44238998 4.44231235
 4.44185196 4.44262394 4.44213279 4.45419877 4.45267903 4.45168402
 4.45111899 4.45053116 4.45011778 4.44935721 4.44889188 4.44825066
 4.44730637 4.44643081 4.44517794 4.4440115 ]

Standard deviation of speed of each vehicle (m/s): 
[1.41977381 1.37045331 1.26624554 1.25935673 1.27420715 1.28121912
 1.28667919 1.29920351 1.2995497  2.28714293 2.2761318  2.26809284
 2.23417767 2.18658328 2.13312659 2.07607496 1.99843891 1.91950613
 1.86415016 1.77048482 1.73174457 1.66270583]

Speed of all vehicles (m/s): Average= 4.45, std=1.73

####################
Total number of vehicles that crossed the zero point: 135

Flow (veh/hour): 1350.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[13.64355954 13.66363328 13.28730844 13.43174989 13.26365099 13.2923708
 13.17290852 13.05310968 12.76006476 13.28074635 13.50491038 13.62283597
 13.62450903 13.9235848 

####################
Acceleration variation (m/s^2): [0.49056660665880786]

File: ./test_time_rollout/ours9x/density_aware_multiagent_ring_20240328-1337431711647463.2143638-0_emission.csv
(9, 3600)
Worst case time headway of controller(s): 
[1.00897294 1.01213864 1.0034282  1.00194048 0.99791805 0.99452632
 0.98972657 0.98186268 1.00321703]


Average time headway of controller(s): 
[1.10241739 1.13897631 1.07254095 1.06033185 1.06086589 1.0555462
 1.05017719 1.06376677 2.96456761]

Standard deviation of time headway of controller(s): 
[0.13663462 0.4363968  0.09839923 0.03054454 0.05621011 0.03258768
 0.0309859  0.0656992  2.2935494 ]

Time headway (s): Worst = 0.98, Avg= 1.29, std= 0.35

####################
####################
Acceleration variation (m/s^2): [0.49056660665880786, 0.483001306313162]

File: ./test_time_rollout/ours9x/density_aware_multiagent_ring_20240328-1337431711647463.4356213-0_emission.csv
(9, 3600)
Worst case time headway of controller(s): 
[1.00899187 1.0126214

In [ ]:
# Data for WAR

In [ ]:
# WAR

### 5. (60%) Multiple Vehicle Control
- 13 vehicles controlled

In [ ]:
# set parameters here
NUM_ROLLOUTS = 5
LENGTH = 260
NUM_CONTROLLED = 13

### FS

In [ ]:
# Change get_desired_velocity velocity to 4.55 instead of 4.82
# Still it can get completely destabilitzed
!python classic.py --method fs --length $LENGTH\
--gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render --num_controlled $NUM_CONTROLLED

In [34]:
# use the eval_metrics code. May have to make 'fs' in tts calculation
!python eval_metrics.py --method fs --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/fs/fs_20240327-1133421711553622.6218832-0_emission.csv', './test_time_rollout/fs/fs_20240327-1136011711553761.6652482-0_emission.csv', './test_time_rollout/fs/fs_20240327-1138361711553916.2422245-0_emission.csv', './test_time_rollout/fs/fs_20240327-1141031711554063.1997359-0_emission.csv', './test_time_rollout/fs/fs_20240327-1143251711554205.6365988-0_emission.csv']

Vehicle ids: ['human_0' 'human_1' 'human_2' 'human_3' 'human_4' 'human_5' 'human_6'
 'human_7' 'human_8' 'fs_0' 'fs_1' 'fs_2' 'fs_3' 'fs_4' 'fs_5' 'fs_6'
 'fs_7' 'fs_8' 'fs_9' 'fs_10' 'fs_11' 'fs_12']

eval_metrics.py:174: RuntimeWarning: divide by zero encountered in true_divide
  time_to_collision = relative_positions/relative_velocities
(13,)
Worst case time to collision for each vehicle (s): 
[-2.13423833 -2.08357439 -2.09784622 -2.20194425 -2.16479081 -2.25979765
 -2.20024496 -2.26577603 -2.29676594 -5.71411685 -7.12334444 -8.94908924
 -3.84790805]

B

Total number of vehicles that crossed the zero point: 97

Flow (veh/hour): 970.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[11.57840651 11.61796306 11.67174122 11.88408102 12.06275167 12.12839152
 12.09766968 11.756326   11.17788393 10.66559823 10.64393838 10.63782157
 10.67335905 10.70771492 10.74086615 10.78729693 10.81290161 10.8547007
 10.87410443 10.838804   10.89681135 10.85499035]

Miles per gallon (mpg): Average= 11.18, std = 0.53

####################
(22, 3600)

Average speed of each vehicle (m/s): 
[4.25054434 4.265813   4.2721989  4.27137983 4.26590585 4.25705658
 4.24576245 4.23287598 4.21967919 4.20643536 4.20140547 4.20770686
 4.22292301 4.23659818 4.25089219 4.25585948 4.25034158 4.24393497
 4.2369194  4.22944444 4.22216855 4.21654407]

Standard deviation of speed of each vehicle (m/s): 
[1.53539031 1.51035181 1.47476681 1.42930278 1.39254414 1.34177766
 1.29357679 1.18755085 1.13223935 1.06560639 1.06747498 1.04355913
 1.01536433 0.9905

####################
Acceleration variation (m/s^2): [0.6032954199704088, 0.333173750173445, 0.606920224965457]

File: ./test_time_rollout/fs/fs_20240327-1143251711554205.6365988-0_emission.csv
(13, 3600)
Worst case time headway of controller(s): 
[1.27129267 1.17097928 1.16716256 1.16719019 1.16689781 1.16706853
 1.16729502 1.16689715 1.16728063 1.16710379 1.16177368 1.16027335
 1.17217198]


Average time headway of controller(s): 
[1.61073347 1.52728708 1.49298115 1.49023368 1.46137094 1.37237753
 1.32186011 1.25904382 1.21682598 1.19998666 1.18905481 1.18580409
 3.54062944]

Standard deviation of time headway of controller(s): 
[0.18764215 0.24331968 0.27733272 0.31458237 0.24023788 0.21809371
 0.18942884 0.11505674 0.08667074 0.08000394 0.07550736 0.07312881
 1.6467764 ]

Time headway (s): Worst = 1.16, Avg= 1.53, std= 0.29

####################
####################
Acceleration variation (m/s^2): [0.6032954199704088, 0.333173750173445, 0.606920224965457, 0.6107478257145863]

#####

In [ ]:
# Data for WAR. Change the desired velocity to 4.55
!python classic.py --method fs --stability --length $LENGTH\
--num_rollouts 5 --gen_emission --render --num_controlled $NUM_CONTROLLED


In [ ]:
# WAR
!python eval_plots.py --method fs 

### PIwS

In [ ]:
!python classic.py --method piws --length $LENGTH\
--gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render --num_controlled $NUM_CONTROLLED


In [36]:
!python eval_metrics.py --method pi --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/pi/piws_20240327-1133441711553624.0477424-0_emission.csv', './test_time_rollout/pi/piws_20240327-1136141711553774.7784283-0_emission.csv', './test_time_rollout/pi/piws_20240327-1138541711553934.7771626-0_emission.csv', './test_time_rollout/pi/piws_20240327-1141331711554093.270436-0_emission.csv', './test_time_rollout/pi/piws_20240327-1144041711554245.0077345-0_emission.csv']

Vehicle ids: ['human_0' 'human_1' 'human_2' 'human_3' 'human_4' 'human_5' 'human_6'
 'human_7' 'human_8' 'piws_0' 'piws_1' 'piws_2' 'piws_3' 'piws_4' 'piws_5'
 'piws_6' 'piws_7' 'piws_8' 'piws_9' 'piws_10' 'piws_11' 'piws_12']

(13,)
Worst case time to collision for each vehicle (s): 
[-21.73330898 -15.84245008 -12.18049081  -5.99614755  -8.33279673
  -7.3511116   -6.81132134  -2.21128541  -1.56861635  -5.22595901
  -3.341603    -1.67711586  -1.45063125]

Best case time to collision for each vehicle (s): 
[ -793888.72352738 -1673062.25094796 -7554

Total number of vehicles that crossed the zero point: 124

Flow (veh/hour): 1240.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[11.89879612 11.72100952 11.61590362 11.53525616 11.35650157 11.06285901
 10.66928927 10.50220454 10.09361283  9.71846661  9.73242169  9.75616927
  9.83140299  9.87108965  9.83491432 10.03103875 10.10104032 10.11231209
 10.01075    10.09026576 10.27815116 10.46489249]

Miles per gallon (mpg): Average= 10.47, std = 0.71

####################
(22, 3600)

Average speed of each vehicle (m/s): 
[3.98116296 3.97936824 3.97772592 3.97629273 3.97495185 3.97345124
 3.9719437  3.97058319 3.96932829 3.96795351 3.96791664 3.96797874
 3.96828208 3.96947123 3.9857282  4.01800108 4.02458837 3.98951648
 3.95251241 3.95104056 3.95070075 3.97380174]

Standard deviation of speed of each vehicle (m/s): 
[1.31434659 1.26571468 1.12006872 1.02648452 0.99971676 0.87698725
 0.81284059 0.72051851 0.56845942 0.308218   0.32499505 0.36638418
 0.4054805  0.4

(13, 3600)
Worst case time headway of controller(s): 
[1.02450477 1.02456683 1.02461443 1.02470959 1.02377055 1.02406125
 1.02304658 1.02341497 1.02174191 1.01732327 1.01954148 1.02245023
 1.01679974]


Average time headway of controller(s): 
[1.25776459 1.27961242 1.31194034 1.38281194 1.51025305 1.662507
 1.72695681 1.78366714 1.71379421 1.68256071 1.82267624 1.99106624
 2.28970976]

Standard deviation of time headway of controller(s): 
[0.49009993 0.53256489 0.57831599 0.63668975 0.76697362 0.89925192
 0.96935173 1.00655901 0.95391785 0.91109808 0.96814843 0.97201636
 0.98595309]

Time headway (s): Worst = 1.02, Avg= 1.65, std= 0.82

####################
####################
Acceleration variation (m/s^2): [0.24635248540454713, 0.2785973846487711, 0.23665745628343057, 0.19347284359740827]

####################
####################

Final Aggregated Safety Metrics (across all files):

Time to Collision across rollouts (s):
	Worst= [-1.450631245652358, -1.3300861270151636, -1.54033592

In [ ]:
# Data for WAR
!python classic.py --method piws --stability --length $LENGTH --num_rollouts 5 --gen_emission\
--render --num_controlled $NUM_CONTROLLED


In [ ]:
# WAR
!python eval_plots.py --method pi 

### BCM

In [ ]:
# For CAV (first get the data)
!python classic.py --method bcm --length $LENGTH\
--num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled $NUM_CONTROLLED


In [37]:
# use the eval_metrics code
!python eval_metrics.py --method bcm --save_plots


Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/bcm/bcm_20240327-1200341711555234.6503558-0_emission.csv', './test_time_rollout/bcm/bcm_20240327-1201291711555289.8281786-0_emission.csv', './test_time_rollout/bcm/bcm_20240327-1202381711555358.2603762-0_emission.csv', './test_time_rollout/bcm/bcm_20240327-1203381711555418.2167916-0_emission.csv', './test_time_rollout/bcm/bcm_20240327-1204451711555485.1878266-0_emission.csv', './test_time_rollout/bcm/bcm_20240327-1205441711555544.7080996-0_emission.csv', './test_time_rollout/bcm/bcm_20240327-1206531711555613.90682-0_emission.csv', './test_time_rollout/bcm/bcm_20240327-1207561711555676.123043-0_emission.csv', './test_time_rollout/bcm/bcm_20240327-1209041711555744.2139525-0_emission.csv', './test_time_rollout/bcm/bcm_20240327-1210161711555816.9564774-0_emission.csv']

Vehicle ids: ['human_0' 'human_1' 'human_2' 'human_3' 'human_4' 'human_5' 'human_6'
 'human_7' 'human_8' 'bcm_0' 'bcm_1' 'bcm_2' 'bcm_3' 'bcm_4' 'bcm_5'
 '

(13,)
Worst case time to collision for each vehicle (s): 
[-134.88669871 -108.34454936  -79.10935761  -55.43710501  -39.54633582
  -27.91795102  -18.94310212  -12.28219739   -7.38514243   -4.78450397
   -3.36058313   -2.48667759   -1.86853551]

Best case time to collision for each vehicle (s): 
[  -393849.86103711  -3546522.99550914  -1994471.15363139
 -38335152.11628064 -10073980.0190154   -9119028.67365303
 -13060706.71620516  -1360405.40902798   -292223.80451929
   -446754.01571729   -332578.88956379   -506756.85639707
  -1039169.8619566 ]

Average time to collision for each vehicle (s): 
[ -4573.1489228  -16662.53253246 -16754.07980545 -49486.90423308
 -24272.28750692 -24075.52012161 -26925.65803767 -10742.26753446
  -4051.81798185  -2906.86988341  -2519.55550227  -2104.91931123
  -1565.58487675]

Standard deviation of time to collision for each vehicle (s): 
[ 16304.37674734 134788.48943357  78209.75867062 997288.1952659
 255700.04980117 265578.12260753 366805.68507883  58201.2770

Total number of vehicles that crossed the zero point: 140

Flow (veh/hour): 1400.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[12.90267887 12.94134681 12.75983926 12.59891276 12.45861419 12.22367519
 12.01552264 11.60948727 11.50291999 11.35517364 11.33282699 11.33271748
 11.33255219 11.33227104 11.3347504  11.45395638 11.4868051  11.5845006
 11.83711616 11.94252173 12.36525351 12.71325241]

Miles per gallon (mpg): Average= 11.93, std = 0.57

####################
(22, 3600)

Average speed of each vehicle (m/s): 
[4.57696718 4.57729841 4.57805086 4.57829836 4.57843057 4.57794757
 4.57768102 4.57774916 4.57780748 4.57778541 4.57775469 4.57772705
 4.5777053  4.57768931 4.57767814 4.57766974 4.5776598  4.57764045
 4.57759934 4.57752004 4.57738696 4.57719981]

Standard deviation of speed of each vehicle (m/s): 
[1.08335278 1.03622519 0.99054115 0.84558152 0.80780693 0.74997695
 0.70137812 0.2714709  0.09607318 0.07265735 0.06558539 0.07022029
 0.08630842 0.11

Total number of vehicles that crossed the zero point: 139

Flow (veh/hour): 1390.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[12.4399117  12.37212442 12.27455869 12.21512565 12.05412705 11.89802075
 11.63756941 11.55394469 11.43228391 11.43240101 11.44161661 11.37893681
 11.37308742 11.37297095 11.37284148 11.37270179 11.37253515 11.37229919
 11.37190131 11.39764083 11.53954816 11.81512788]

Miles per gallon (mpg): Average= 11.66, std = 0.37

####################
(22, 3600)

Average speed of each vehicle (m/s): 
[4.59260234 4.59280712 4.59328457 4.59357445 4.59324794 4.5929031
 4.59212162 4.59229548 4.59211502 4.59216494 4.59219506 4.59219821
 4.59218132 4.59215421 4.59212449 4.59209694 4.59207465 4.59206041
 4.5920587  4.59207908 4.59214231 4.59229024]

Standard deviation of speed of each vehicle (m/s): 
[0.50106396 0.48451956 0.46791331 0.45793046 0.399926   0.3846133
 0.13904083 0.14106882 0.15284032 0.09258366 0.06036036 0.04208525
 0.03305578 0.032

Standard deviation of time headway of controller(s): 
[0.00673642 0.01091012 0.01728769 0.02423138 0.03178846 0.04048221
 0.0512257  0.06542869 0.08499445 0.10298545 0.12430562 0.14515315
 0.16699593]

Time headway (s): Worst = 1.04, Avg= 1.5, std= 0.07

####################
####################
Acceleration variation (m/s^2): [0.1261974677178484, 0.2268232917985134, 0.21470794379149277, 0.16777982988827977, 0.22535883400072274, 0.1646687956234412]

File: ./test_time_rollout/bcm/bcm_20240327-1207561711555676.123043-0_emission.csv
(13, 3600)
Worst case time headway of controller(s): 
[1.39438458 1.33896529 1.28244156 1.22144035 1.1482071  1.04886742
 1.03299981 1.0324788  1.03271854 1.03142658 1.03169339 1.02856652
 1.02336703]


Average time headway of controller(s): 
[1.4410989  1.4448126  1.44854912 1.45228002 1.45594996 1.45945131
 1.46420862 1.47045907 1.47953592 1.49113835 1.50640831 1.52563885
 1.54826176]

Standard deviation of time headway of controller(s): 
[0.01160358 0.02433

In [ ]:
# Data for WAR
!python classic.py --method bcm --stability --length $LENGTH --gen_emission\
--num_rollouts 5 --render --num_controlled $NUM_CONTROLLED


In [ ]:
# WAR
!python eval_plots.py --method bcm 

### LACC

In [ ]:
# For CAV (first get the data)
!python classic.py --method lacc --length $LENGTH\
--num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled $NUM_CONTROLLED --shock_start_time 11400 --shock_end_time 15000


In [38]:
# use the eval_metrics code
!python eval_metrics.py --method lacc --start_time 11400 --end_time 15000 --save_plots


Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/lacc/lacc_20240327-1222281711556548.4627247-0_emission.csv', './test_time_rollout/lacc/lacc_20240327-1224491711556689.7502203-0_emission.csv', './test_time_rollout/lacc/lacc_20240327-1227121711556832.5814385-0_emission.csv', './test_time_rollout/lacc/lacc_20240327-1229371711556977.8224914-0_emission.csv', './test_time_rollout/lacc/lacc_20240327-1231591711557119.4821625-0_emission.csv']

Vehicle ids: ['human_0' 'human_1' 'human_2' 'human_3' 'human_4' 'human_5' 'human_6'
 'human_7' 'human_8' 'lacc_0' 'lacc_1' 'lacc_2' 'lacc_3' 'lacc_4' 'lacc_5'
 'lacc_6' 'lacc_7' 'lacc_8' 'lacc_9' 'lacc_10' 'lacc_11' 'lacc_12']

(13,)
Worst case time to collision for each vehicle (s): 
[-24.38152851 -21.82751834 -19.46702088 -17.28513739 -15.26409319
 -13.3857504  -11.6419465  -10.01883295  -8.50002678  -7.08109554
  -5.74594924  -4.4896651   -3.16936248]

Best case time to collision for each vehicle (s): 
[-1205083.04540139  -118301.709

Total number of vehicles that crossed the zero point: 146

Flow (veh/hour): 1460.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[14.29893211 14.25222345 13.96989944 13.53037803 13.55661115 13.64684929
 13.57910929 13.37337115 13.09066293 12.83992633 12.93187759 13.06390236
 13.1681708  13.24790136 13.32141522 13.46594068 13.58897405 13.69423832
 13.88973602 14.07219087 14.28528769 14.31020665]

Miles per gallon (mpg): Average= 13.6, std = 0.45

####################
(22, 3600)

Average speed of each vehicle (m/s): 
[4.7027788  4.70550591 4.70178318 4.69880595 4.69671469 4.695319
 4.69511909 4.69601306 4.69784979 4.70071907 4.70365875 4.70758205
 4.71216584 4.71698609 4.72155361 4.72536542 4.72796438 4.72899604
 4.72825046 4.72566674 4.72122184 4.71453055]

Standard deviation of speed of each vehicle (m/s): 
[1.38581052 1.28111906 1.20046325 1.13354493 1.16271677 1.13206105
 1.09432084 1.04983107 0.73432914 0.71513618 0.74477474 0.77749381
 0.81299694 0.8507

(13, 3600)
Worst case time headway of controller(s): 
[1.39379607 1.39334693 1.39283867 1.39225903 1.391591   1.39081311
 1.38989659 1.3888017  1.38746816 1.38582372 1.38373783 1.3811336
 1.37743089]


Average time headway of controller(s): 
[1.40013748 1.40015017 1.40016261 1.40017502 1.40018781 1.4002017
 1.40021767 1.40023707 1.4002616  1.40029339 1.40033532 1.40039174
 1.40046901]

Standard deviation of time headway of controller(s): 
[0.00302723 0.00319031 0.00336613 0.00355665 0.00376414 0.0039914
 0.00424205 0.00452114 0.00483617 0.00519884 0.00562868 0.00616156
 0.00687524]

Time headway (s): Worst = 1.38, Avg= 1.4, std= 0.0

####################
####################
Acceleration variation (m/s^2): [0.2214128501683324, 0.36578474244486037, 0.29359237051040277, 0.32662100918200304]

####################
####################

Final Aggregated Safety Metrics (across all files):

Time to Collision across rollouts (s):
	Worst= [-3.1693624821889124, -1.0502475223715233, -4.0112974349

In [ ]:
# Data for WAR
!python classic.py --method lacc --stability --length $LENGTH --num_rollouts 5 --gen_emission\
--render --num_controlled $NUM_CONTROLLED --shock_start_time 11400 --shock_end_time 15000

In [ ]:
# WAR
!python eval_plots.py --method lacc --start_time 11400 --end_time 15000 --propogate_time 200

### Wu et. al. (60%)

In [ ]:
# Agents are supposed to be dispersed. Exp configs folder should be present outside where test_rllib.py is present
# Stabalization time looks high, shocks can start later
# 90 is highest but cannot stabilize at 90
!python test_rllib.py ./Wu_et_al/Trained_policies/60_percent/PPO_MultiAgentWaveAttenuationPOEnv-v1_24561808_2024-02-15_12-53-36bmm0ybn0 130\
--method wu --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled $NUM_CONTROLLED


# !python test_rllib.py ./Wu_et_al/Trained_policies/60_percent/PPO_MultiAgentWaveAttenuationPOEnv-v1_31346110_2024-02-15_12-53-57as6iehau 90\
# --method wu --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled $NUM_CONTROLLED



In [39]:
!python eval_metrics.py --method wu --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/wu/multiagent_ring_20240327-1237051711557425.4694579-0_emission.csv', './test_time_rollout/wu/multiagent_ring_20240327-1241201711557680.1414876-0_emission.csv', './test_time_rollout/wu/multiagent_ring_20240327-1245331711557933.607279-0_emission.csv', './test_time_rollout/wu/multiagent_ring_20240327-1249531711558193.747305-0_emission.csv', './test_time_rollout/wu/multiagent_ring_20240327-1254111711558451.8689-0_emission.csv']

Vehicle ids: ['rl_0_0' 'human_0_0' 'rl_1_0' 'human_1_0' 'rl_2_0' 'human_2_0' 'rl_3_0'
 'human_3_0' 'rl_4_0' 'human_4_0' 'rl_5_0' 'rl_6_0' 'human_6_0' 'rl_7_0'
 'rl_8_0' 'human_8_0' 'rl_9_0' 'rl_10_0' 'human_10_0' 'rl_11_0' 'rl_12_0'
 'human_12_0']

eval_metrics.py:174: RuntimeWarning: divide by zero encountered in true_divide
  time_to_collision = relative_positions/relative_velocities
eval_metrics.py:174: RuntimeWarning: overflow encountered in true_divide
  time_to_collision = relative_positions

(22, 3600)

Average speed of each vehicle (m/s): 
[1.06888951 1.05798696 1.04826288 1.05867713 1.04993282 1.05814912
 1.04994908 1.04282388 1.03309902 1.0259897  1.01672171 1.00519745
 0.99404099 0.98463384 1.00689516 0.99744103 0.98927968 0.98297635
 1.01727976 1.00954206 1.02812853 1.07915448]

Standard deviation of speed of each vehicle (m/s): 
[1.17714315 1.1591361  1.15166416 1.16150513 1.15416062 1.12551909
 1.1179192  1.10894997 1.10088446 1.0838867  1.07510181 1.0845268
 1.02794464 1.01672457 1.00300117 0.9620445  0.95095097 0.9454162
 1.28552409 1.23194195 1.19587534 1.18570635]

Speed of all vehicles (m/s): Average= 1.03, std=1.1

####################
Total number of vehicles that crossed the zero point: 30

Flow (veh/hour): 300.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[5.10305427 5.33772263 5.18631906 5.28601176 5.09442516 5.25125289
 5.10776367 5.31852867 5.12144017 5.35488079 5.18133885 5.21855752
 5.39123187 5.12863368 5.07920159 5.2651

####################
Acceleration variation (m/s^2): [0.2579874450272726, 0.10446806382672001, 0.05916536723026434]

File: ./test_time_rollout/wu/multiagent_ring_20240327-1254111711558451.8689-0_emission.csv
(13, 3600)
Worst case time headway of controller(s): 
[1.83852251 1.75905221 1.78675585 1.79402284 1.77952753 1.78359421
 1.79193689 1.78818371 1.78993664 1.83872205 1.84122396 1.86072637
 1.85095246]


Average time headway of controller(s): 
[5.43097036 4.31501182 2.59120893 2.72827447 4.16489488 2.7205145
 3.41964716 2.73664932 3.90386292 2.79962853 2.44166592 3.05426663
 2.58097207]

Standard deviation of time headway of controller(s): 
[4.77379908 2.62259162 0.55441753 1.65405467 1.80352398 0.55213742
 1.33738975 0.86550601 2.01002256 0.88220545 0.41058417 1.72442313
 0.68613876]

Time headway (s): Worst = 1.76, Avg= 3.3, std= 1.53

####################
####################
Acceleration variation (m/s^2): [0.2579874450272726, 0.10446806382672001, 0.05916536723026434, 0.24025441

In [ ]:
# Data for WAR

In [ ]:
# WAR

### Ours (60%)

In [ ]:
#### Safety + Stability ####
# For CAV (first get the data). Copy the exp_configs to a folder where test_rllib.py is located
# Make sure efficiency is set to False in controller
# Make sure the estimate is turned ON in multiagent env (in 2 places)
!python test_rllib.py ./Ours/Trained_policies/60_percent/PPO_MultiAgentDensityAwareRLEnv-v1_18eaa6f2_2024-02-16_15-29-348o30nqxq 200\
--method ours13x --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled $NUM_CONTROLLED


In [ ]:
!python eval_metrics.py --method ours13x --save_plots

In [ ]:
#### Efficiency ####
# Make sure to set the efficiency to true at test time in the controller file and have those files at exp_configs
# Make sure to have the right leader
!python test_rllib.py ./Ours/Trained_policies/60_percent/PPO_MultiAgentDensityAwareRLEnv-v1_ed546dfe_2024-02-14_17-18-06nijs5agx 200\
--method ours13x --length $LENGTH --num_rollouts 20 --render --gen_emission --shock --num_controlled $NUM_CONTROLLED


In [41]:
!python eval_metrics.py --method ours13x --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/ours13x/density_aware_multiagent_ring_20240327-1316081711559768.8918848-0_emission.csv', './test_time_rollout/ours13x/density_aware_multiagent_ring_20240327-1316461711559806.2174933-0_emission.csv', './test_time_rollout/ours13x/density_aware_multiagent_ring_20240327-1321151711560075.9599729-0_emission.csv', './test_time_rollout/ours13x/density_aware_multiagent_ring_20240327-1321561711560116.2070596-0_emission.csv', './test_time_rollout/ours13x/density_aware_multiagent_ring_20240327-1326351711560395.7169347-0_emission.csv', './test_time_rollout/ours13x/density_aware_multiagent_ring_20240327-1327191711560439.4033725-0_emission.csv', './test_time_rollout/ours13x/density_aware_multiagent_ring_20240327-1332081711560728.7980657-0_emission.csv', './test_time_rollout/ours13x/density_aware_multiagent_ring_20240327-1332531711560773.642653-0_emission.csv', './test_time_rollout/ours13x/density_aware_multiagent_ring_20240327-133736

(13,)
Worst case time to collision for each vehicle (s): 
[-15.50760101 -13.65324994 -16.28117484 -15.80613826 -14.29103383
 -18.7352925   -9.75912767 -13.08019996 -15.43994516 -15.58929651
 -11.8320099  -24.07964604  -3.97311884]

Best case time to collision for each vehicle (s): 
[  -118275.56435802    -65926.82651079    -86873.00418441
    -59970.45656767   -377176.14869698    -73604.10062973
    -39214.44180334   -216828.14576816    -43103.89150301
    -51842.51413509   -570743.43317941 -10377249.39225065
   -508638.08051416]

Average time to collision for each vehicle (s): 
[ -349.97939493  -355.07632179  -407.82379421  -397.43155815
  -573.39428592  -311.50662074  -256.3077372   -458.16998793
  -300.46291482  -388.67085097  -788.46688629 -5935.70649002
  -471.13214964]

Standard deviation of time to collision for each vehicle (s): 
[  3138.78943475   1882.33309348   2691.34160975   2650.37626012
   9204.70461504   2005.65708154   1342.65879958   5322.68714151
   1526.91701734   1

(22, 3600)

Average speed of each vehicle (m/s): 
[4.32630568 4.32574004 4.32561691 4.3256117  4.32539149 4.32495373
 4.32302584 4.32188673 4.32030286 4.32000053 4.32024674 4.31996433
 4.32002085 4.3287283  4.32871253 4.32869723 4.3285013  4.32826705
 4.32776661 4.32729276 4.32681932 4.32653332]

Standard deviation of speed of each vehicle (m/s): 
[0.16886548 0.16835247 0.16909161 0.17288602 0.17630743 0.17489781
 0.17298294 0.17254678 0.17145254 0.16946175 0.16782976 0.16427389
 0.15432329 1.03219205 1.02237689 1.01142168 0.89809523 0.88180613
 0.74355586 0.65376088 0.51200107 0.36988086]

Speed of all vehicles (m/s): Average= 4.33, std=0.42

####################
Total number of vehicles that crossed the zero point: 131

Flow (veh/hour): 1310.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[12.63012058 12.64421912 12.66627878 12.83840967 12.88364898 12.88910232
 13.1291138  13.3058811  13.15533863 13.02856108 12.99319835 12.77973921
 10.97631978 14.0636044

(22, 3600)

Average speed of each vehicle (m/s): 
[4.26171545 4.26211897 4.26269403 4.26358049 4.26398441 4.26482222
 4.26525047 4.26492742 4.26602833 4.26696214 4.26818858 4.26859051
 4.26945746 4.25816219 4.25842629 4.25899275 4.2594369  4.2600848
 4.26052831 4.26096135 4.2612548  4.26159438]

Standard deviation of speed of each vehicle (m/s): 
[0.11823561 0.11685511 0.12034499 0.12497213 0.12652192 0.12824582
 0.13184593 0.1338244  0.13360029 0.13213568 0.13522287 0.12863891
 0.11602506 0.8441755  0.76802366 0.65075231 0.45191681 0.38008392
 0.35986533 0.172768   0.18234    0.14950677]

Speed of all vehicles (m/s): Average= 4.26, std=0.25

####################
Total number of vehicles that crossed the zero point: 130

Flow (veh/hour): 1300.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[12.38636047 12.34368649 12.49644073 12.63788371 12.58000995 12.5020405
 12.80177173 12.86313905 12.89418568 12.61126446 12.73735003 12.63081528
 10.71835439 12.64794015 

####################
Acceleration variation (m/s^2): [0.40146874884618367, 0.38945617864314164, 0.4171970412527972, 0.41674717394168415, 0.3990044848265382, 0.4054978325837077]

File: ./test_time_rollout/ours13x/density_aware_multiagent_ring_20240327-1332531711560773.642653-0_emission.csv
(13, 3600)
Worst case time headway of controller(s): 
[1.02257048 1.0206814  1.01959823 1.02181297 1.01950968 1.02158019
 1.02077975 1.02170872 1.02037748 1.01579196 1.0128102  1.00952454
 1.03216342]


Average time headway of controller(s): 
[1.06597779 1.05427435 1.056295   1.06160272 1.06326071 1.06030291
 1.06695366 1.07411167 1.06688073 1.06399675 1.06402785 1.05868622
 7.62492256]

Standard deviation of time headway of controller(s): 
[0.03009144 0.02139453 0.02555646 0.02606863 0.02697863 0.02567381
 0.02927089 0.03083874 0.02830534 0.02891314 0.02695824 0.02518583
 2.27588372]

Time headway (s): Worst = 1.01, Avg= 1.57, std= 0.2

####################
####################
Acceleration variation

In [ ]:
# Data for WAR

In [ ]:
# WAR

## C. Data collection for Stability Plot
- Execute the `eval_plots.py` code as well because this will create the stability plots (sometimes manual inspection is required)

In [ ]:
LENGTH_STABILITY = 270 #m
NUM_ROLLOUTS_STABILITY = 1

### IDM

In [ ]:
# This is for plot
# Stability shock is applied to IDM when the system is stable and SGT is not formed yet
# 150 is pretty early-on
!python classic.py --method idm --length $LENGTH_STABILITY --gen_emission --stability --shock_start_time 150 --shock_end_time 3710 --render

In [ ]:
# IDM is unstable (So there is no wave attenuation) - WAR does not apply
# But do we still need a value for reference (WAR = 0)
!python eval_plots.py --method idm --start_time 150 --end_time 3710 

### FS

In [ ]:
!python classic.py --method fs --stability --length $LENGTH_STABILITY --num_rollouts $NUM_ROLLOUTS_STABILITY --gen_emission --render

In [ ]:
!python eval_plots.py --method fs 

### PIwS

In [ ]:
# Data for WAR (Check if it stabilized or not)
!python classic.py --method piws --stability --length $LENGTH_STABILITY --num_rollouts $NUM_ROLLOUTS_STABILITY --gen_emission --render
# Check if was stable or not before the shock was applied

In [ ]:
!python eval_plots.py --method pi 

### RL+ LO (Wu et al.)

In [ ]:
!python test_rllib.py ./Wu_et_al/Trained_policies/PPO_WaveAttenuationPOEnv-v0_25b5cb6e_2022-01-26_10-58-12e9f4i3ao 50\
--method wu --shock --stability --length $LENGTH_STABILITY --gen_emission --num_rollouts $NUM_ROLLOUTS_STABILITY --render


In [ ]:
!python eval_plots.py --method wu 

### Ours (5%)
- Set local zone to 35m in the file before executing the code below

In [ ]:
# This data is taken with the local zone set to 35m

# Data for WAR (Take the Safety + Stability Policy)
!python test_rllib.py ./Ours/Trained_policies/5_percent/PPO_DensityAwareRLEnv-v0_5dfded14_2024-02-07_09-58-351869u4p3 168\
--method ours --shock --stability --length $LENGTH_STABILITY --gen_emission --num_rollouts 1 --render --num_controlled 1


In [ ]:
!python eval_plots.py --method ours

### BCM

In [ ]:
!python classic.py --method bcm --stability --length $LENGTH_STABILITY --num_rollouts $NUM_ROLLOUTS_STABILITY --gen_emission --num_rollouts $NUM_ROLLOUTS_WAR --render


In [ ]:
!python eval_plots.py --method bcm 

### Ours (20%)
- Set Local zone to 40m in the file before executing the code below 

In [ ]:
# This data is taken with the local zone set to 40m
# For CAV (first get the data). Copy the exp_configs to a folder where test_rllib.py is located
# Make sure efficiency is set to False in controller
# Make sure the estimate is turned ON in multiagent env (in 2 places)
# Data for WAR (taken on the Safety + Stability model). Make sure everything is tuned for Safety + Stability

!python test_rllib.py ./Ours/Trained_policies/20_percent/PPO_MultiAgentDensityAwareRLEnv-v1_afca7452_2024-02-11_14-35-18pt1771a1 140\
--method ours4x --shock --stability --length $LENGTH_STABILITY --gen_emission --num_rollouts 1 --render --num_controlled 4


In [ ]:
!python eval_plots.py --method ours4x

### LACC

In [ ]:
!python classic.py --method lacc --stability --length $LENGTH_STABILITY --num_rollouts $NUM_ROLLOUTS_STABILITY --gen_emission --shock_start_time 11400 --shock_end_time 15000 --render

In [ ]:
!python eval_plots.py --method lacc --start_time 11400 --end_time 15000 --propogate_time 200